# Map of earthquakes and faults in Southern California

# 1. Load in packages and data

## 1.0 Packages

In [ ]:
import sys, os
sys.path.append(f"../../seispy")
import seispy
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import geodatasets
from shapely.wkt import loads
import zipfile 
import fiona 
# from bs4 import BeautifulSoup
# import osgeo
# import rasterio
import contextily as cx
from shapely.geometry import Point
import seaborn as sns
import xyzservices.providers as xyz
from IPython.display import clear_output
import numpy as np
import pyproj
from obspy.imaging.beachball import beach
from matplotlib.text import OffsetFrom
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib.markers import MarkerStyle
from pyproj import Transformer
import math
import string
import ast
from pathlib import Path
import shutil

alphabet = string.ascii_lowercase

fiona.drvsupport.supported_drivers['libkml'] = 'rw' 
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

source = cx.providers.Esri.WorldTerrain

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('axes', labelsize=20)
rc_params_dict = {'axes.titlesize': 20, 'xtick.labelsize':20, 'ytick.labelsize':20, 'legend.fontsize':20}
plt.rcParams.update(rc_params_dict)

scale_eq_marker = (lambda x: 10 + np.exp(1.1*x))

# plot_colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e', '#969696', '#000000']
plot_colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d','#666666']
plot_color_dict = dict(zip(['teal', 'orange', 'purple', 'pink', 'green', 'yellow', 'brown', 'grey'], plot_colors))
plot_color_dict

def lat_lon_tick_labels(lon, lat, ax, fontsize=20):
    transformer_4326_to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    x_ticks_3857 = [transformer_4326_to_3857.transform(lat[0], x)[0] for x in lon]
    y_ticks_3857 = [transformer_4326_to_3857.transform(y, lon[0])[1] for y in lat]

    ax.set_xticks(x_ticks_3857)
    ax.set_yticks(y_ticks_3857)
    ax.set_xticklabels(lon, fontsize=fontsize)
    ax.set_yticklabels(lat, fontsize=fontsize)

def lon_lat_to_geometry(lon, lat):
    geometry = [Point(xy) for xy in zip(lon, lat)]
    gdf = gpd.GeoDataFrame(geometry=geometry)
    gdf = gdf.set_crs(epsg=4326)
    gdf = gdf.to_crs(epsg=3857)
    return gdf

def convert_extent_to_epsg3857(extent):
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
    x_min, y_min = transformer.transform(extent[0], extent[2])
    x_max, y_max = transformer.transform(extent[1], extent[3])
    return [x_min, x_max, y_min, y_max]

def add_box_3857(rectangle_coords, ax, color='red', zorder=0, linewidth=1, alpha=0.9):
    rectangle_coords = convert_extent_to_epsg3857(rectangle_coords)

    rectangle = Rectangle(
        (rectangle_coords[0], rectangle_coords[2]),
        rectangle_coords[1] - rectangle_coords[0],
        rectangle_coords[3] - rectangle_coords[2],
        linewidth=linewidth, edgecolor=color, facecolor='none', zorder=zorder, alpha=alpha
    )
    ax.add_patch(rectangle)

def df_to_gdf(df, inplace=False):
    gdf = gpd.GeoDataFrame(df,
                           geometry=gpd.points_from_xy(df.LON, df.LAT),
                           crs="EPSG:4326")
    gdf.to_crs(epsg=3857, inplace=True)

    if inplace==True:
        df = gdf.copy()
    elif inplace==False:
        return gdf

def EPSG_transformer(points, current=4326, new=3857):
    transformer = Transformer.from_crs(f"EPSG:{current}", f"EPSG:{new}")
    
    if isinstance(points, tuple):
        points = [points]
    elif isinstance(points, np.ndarray):
        points = points.reshape(-1, 2)
    
    transformed_points = [transformer.transform(lat, lon) for lon, lat in points]
    return transformed_points

def plot_SCSN_reporting_region(ax, color=plot_color_dict['green'], linewidth=1, zorder=0, alpha=0.9):
    reporting_region = [[-117.76, 37.43], 
                    [-114, 34.5],
                    [-114, 31.5],
                    [-118.5, 31.5],
                    [-121.25, 34.5],
                    [-117.76, 37.43]
                    ]
    geometry = [Point(p) for p in reporting_region]
    reporting_region_gdf = gpd.GeoDataFrame(geometry=geometry)
    reporting_region_gdf.set_crs(epsg=4326, inplace=True)
    reporting_region_gdf.to_crs(epsg=3857, inplace=True)
    reporting_region_gdf

    # ax = reporting_region_gdf.plot()
    ax.plot(reporting_region_gdf['geometry'].x, reporting_region_gdf['geometry'].y, color=color, linewidth=linewidth, zorder=zorder, alpha=alpha)

def Leonard_rupture_length(magnitudes):
    """
    Leonard (2010) magnitude-rupture (half) length relation.
    """
    a_mw = 1.67
    b_mw = 4.17
    magnitudes = np.array(magnitudes)
    rupture_length = 10**((magnitudes - b_mw)/a_mw)
    return rupture_length

def string_to_datetime_df(dataframe, format='%Y-%m-%d %H:%M:%S.%f'):
    """
    Find DATETIME column in df and change to datetime objects
    """
    dataframe['DATETIME'] = pd.to_datetime(dataframe['DATETIME'],
                                           format = format)

# 2. Preprocessing data

## 2.2 SCEC fault  data
https://www.scec.org/research/cfm

In [ ]:
fault_gdf = gpd.read_file("../../catalogues/SCEC/CFM6/CFM6.1_traces.kml", driver='libkml')
fault_gdf.set_crs(epsg=4326, inplace=True)
fault_gdf.to_crs(epsg=3857, inplace=True)
fault_gdf

In [ ]:
ax = fault_gdf.plot()
cx.add_basemap(ax, source=source)

In [ ]:
grouped = fault_gdf.groupby('Name')

segment_lengths = []

for name, group in grouped:
    centroids = group['geometry'].centroid

    max_y = centroids.y.max()
    min_y = centroids.y.min()

    length = max_y - min_y

    segment_lengths.append((name, length))

segment_lengths.sort(key=lambda x: x[1], reverse=True)

for name, length in segment_lengths[:10]:
    print(f"Name: {name}, Length: {length}")

names = [segment[0] for segment in segment_lengths[:10]]
names

### 1.2.1 Adding in station data
https://service.scedc.caltech.edu/station/weblist.php  
Had to read into excel and then resave as a csv

In [ ]:
# station_df = pd.read_csv('../../catalogues/SCEC/stations.csv', delim_whitespace=True, on_bad_lines='skip')
station_df = pd.read_csv('../../catalogues/SCEC/stations_imported.csv')
station_df = df_to_gdf(station_df)
station_df

In [ ]:
ax = station_df['geometry'].plot(marker='v', color='green', markersize=0.5)
cx.add_basemap(source=source, ax=ax)
big_map_extent = convert_extent_to_epsg3857([-122, -114.5, 31.2, 37.3])
ax.axis(big_map_extent)

# 3. Load in earthquake data and convert to geometry data

In [ ]:
# catalogue_names =['QTM_9_5', 'QTM_12', 'SCSN', 'HYS']
catalogue_names =['SCSN', 'QTM_12', 'QTM_9_5']
region = [-118.80, -115.40, 32.68, 36.20]

start_date = dt.datetime(2008, 1, 1)
end_date = dt.datetime(2018, 1, 1)
catalogue_dict = {}
for name in catalogue_names:
    catalogue_file = pd.read_csv(f"../../catalogues/reformatted/{name}_reformat.csv")
    string_to_datetime_df(catalogue_file)
    # catalogue_file = catalogue_file.loc[(catalogue_file['DATETIME'] >= start_date) &\
    #                                     (catalogue_file['DATETIME'] < end_date)].copy()
    # catalogue_file.to_csv(f"../../catalogues/reformatted/{name}_reformat.csv", index=False)
    catalogue_file = df_to_gdf(catalogue_file)
    catalogue_dict.update({name:catalogue_file})

mainshock_dict = {}
for name, data in catalogue_dict.items():
    mainshock_file = pd.read_csv(f'../data/{name}/mainshocks.csv')
    string_to_datetime_df(mainshock_file)
    mainshock_file['YEAR'] = seispy.datetime_to_decimal_year(mainshock_file['DATETIME'])
    # mainshock_file.to_csv(f'../data/{name}/mainshocks.csv', index=False)
    mainshock_file = df_to_gdf(mainshock_file)
    mainshock_dict[name] = mainshock_file

QTM_9_5 = catalogue_dict['QTM_9_5'].copy()
QTM_12 = catalogue_dict['QTM_12'].copy()
SCSN = catalogue_dict['SCSN'].copy()

QTM_9_5_mainshocks = mainshock_dict['QTM_9_5'].copy()
QTM_12_mainshocks = mainshock_dict['QTM_12'].copy()
SCSN_mainshocks = mainshock_dict['SCSN'].copy()
# HYS = catalogue_dict['HYS'].copy()

mainshocks_in_all_QTM_9_5 = QTM_9_5_mainshocks.loc[(QTM_9_5_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
                                                        (QTM_9_5_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

mainshocks_in_all_QTM_12 = QTM_12_mainshocks.loc[(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                                        (QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID']))].copy()

mainshocks_in_all_SCSN = SCSN_mainshocks.loc[(SCSN_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                                        (SCSN_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

mainshocks_in_all_dict = {'QTM_9_5':mainshocks_in_all_QTM_9_5,
                          'QTM_12':mainshocks_in_all_QTM_12,
                          'SCSN':mainshocks_in_all_SCSN}

QTM_9_5_only_mainshocks = QTM_9_5_mainshocks.loc[~(QTM_9_5_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
                                                 ~(QTM_9_5_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

QTM_12_only_mainshocks = QTM_12_mainshocks.loc[~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
                                               ~(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID']))].copy  ()

SCSN_only_mainshocks = SCSN_mainshocks.loc[~(SCSN_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                           ~(SCSN_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

QTM_only_mainshocks =  QTM_12_mainshocks.loc[(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                             ~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID']))].copy()

merged_mainshocks_catalog = pd.merge(pd.merge(mainshocks_in_all_SCSN, mainshocks_in_all_QTM_12, on=['ID', 'Selection'], how='inner', suffixes=('', '_QTM_12')), mainshocks_in_all_QTM_9_5, on=['ID', 'Selection'], how='inner', suffixes=('', '_QTM_9_5'))

DDET_mainshocks = merged_mainshocks_catalog.loc[merged_mainshocks_catalog['Selection']=='Both'].copy()

# merged_mainshocks_catalog = pd.read_csv('../data/mainshocks/merged_mainshock_catalog.csv')
merged_mainshocks_catalog.to_csv('../data/mainshocks/merged_mainshock_catalog.csv', index=False)
seispy.string_to_datetime_df(merged_mainshocks_catalog)

good_mainshocks = pd.read_csv('../outputs/good_mainshocks.csv')
seispy.string_to_datetime_df(good_mainshocks, format="%Y-%m-%d %H:%M:%S.%f")

geometry_dict = {}
for key, item in catalogue_dict.items():
    geometry = [Point(xy) for xy in zip(item['LON'], item['LAT'])]
    gdf = gpd.GeoDataFrame(geometry=geometry)
    gdf = gdf.set_crs(epsg=4326)
    # gdf = gdf.set_crs(epsg=3857)
    geometry_dict.update({key:gdf})

CNN_SoCal = pd.read_csv('../../catalogues/reformatted/CNN_SoCal_reformat.csv')
string_to_datetime_df(CNN_SoCal)
largest_events = CNN_SoCal.loc[CNN_SoCal['MAGNITUDE']>=7].copy()
largest_events['NAME'] = ['Landers 1992', 'Hector-Mine 1999', 'El-Mayor Cucupah 2010', 'Ridgecrest 2019']
largest_events_gdf = gpd.GeoDataFrame(
    largest_events,
    geometry=gpd.points_from_xy(largest_events.LON, largest_events.LAT),
    crs="EPSG:4326"  # WGS84 Latitude/Longitude
)

largest_events_gdf = largest_events_gdf.to_crs(epsg=3857)
largest_events_gdf

fault_gdf = gpd.read_file("../../catalogues/SCEC/CFM6/CFM6.1_traces.kml", driver='libkml')
fault_gdf.set_crs(epsg=4326, inplace=True)
fault_gdf.to_crs(epsg=3857, inplace=True)
fault_gdf

In [ ]:
good_mainshocks

# 2. Plot data fault data with earthquakes and focal mechanisms

## 2.1 Testing area
This section got messy, just try and make the plots you want to make look nice, stop trying to optimise.

In [ ]:
bbox_args = dict(boxstyle="round,pad=0.5", fc='white', alpha=1)
neither_relation = (lambda x: scale_eq_marker(x)/2)

plot_options_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['grey'], 'size':neither_relation, 'linewidth':0.25}
                    }

cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':scale_eq_marker},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':scale_eq_marker},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':scale_eq_marker},
                    }

scale = 2
fontsize=20*scale
size_dict = {'figsize_x': 8*scale, 'figsize_y': 8*scale,
             'textsize': fontsize,'linewidth':1*scale,
             'legend_markersize':fontsize/2, 'neither_markersize':fontsize/4,
             'axes.titlesize': fontsize, 'xtick.labelsize':fontsize, 
             'ytick.labelsize':fontsize, 'legend.fontsize':fontsize
             }

rc_params_dict = {'axes.titlesize': size_dict['textsize'], 
                  'xtick.labelsize':size_dict['textsize']*scale,
                  'ytick.labelsize':size_dict['textsize']*scale,
                  'legend.fontsize':size_dict['textsize']*scale/6}

### 2.1.0 Rabbit holing on making the size of my markers equivalent to the rupture radius

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
x = [10, 20, 25, 30, 35]
y = [10, 20, 25, 30, 35]

# Different radii for markers in meters
radii = [3, 4, 5, 6, 7]  # Radii in meters

# Conversion factor to translate meters to points squared
# Assuming figure size is (10, 10) inches and plot extent is 40 meters
fig_size_inch = 10
extent_meters = 40
meters_to_points = fig_size_inch * 63 / extent_meters  # 72 points per inch

# Calculate the areas in points squared
sizes = [np.pi * (radius * meters_to_points) ** 2 for radius in radii]

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

for i in range(len(x)):
    circle = Circle((x[i], y[i]), radii[i], alpha=0.6, edgecolor='b', facecolor='b')
    ax.add_patch(circle)
    ax.text(x[i], y[i], f'r={radii[i]}m', fontsize=12, ha='right')

# Create scatter plot
ax.scatter(x, y, s=sizes, alpha=0.6, fc='r', ec='r')

# Annotate each point with its corresponding radius
for i, radius in enumerate(radii):
    ax.text(x[i], y[i], f'r={radius}m', fontsize=12, ha='right')

# Set limits to match the extent in meters
ax.set_xlim(0, 40)
ax.set_ylim(0, 40)
ax.set_xlabel('X axis (meters)')
ax.set_ylabel('Y axis (meters)')
# ax.title('Scatter Plot with Marker Sizes Based on Radius in Meters')

# Show grid
ax.grid(True)

plt.show()


In [ ]:
largest_events_gdf['rupture_length'] = Leonard_rupture_length(largest_events_gdf['MAGNITUDE'])
largest_events_gdf

In [ ]:
big_map_extent

In [ ]:
x = np.linspace(-13403241, -12857026.103195205, 5)
x = np.append(x, x)
x

In [ ]:
y = [3865259.1407130747]*5
y.extend([4209031.393076654]*5)
y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# x = np.array(largest_events_gdf['geometry'].x)
# y = np.array(largest_events_gdf['geometry'].y)
# sample_points = SCSN.sample(6)
# x = np.array(sample_points['geometry'].x)
# y = np.array(sample_points['geometry'].y)

# Different radii for markers in meters
# radii = np.array(largest_events_gdf['rupture_length']*1000)
magnitudes = np.array([4, 4.5, 4.9, 5.1, 5.8, 6.2, 6.5, 6.8, 7, 7.2])
lengths = Leonard_rupture_length(magnitudes)
radii = lengths*1000

# Conversion factor to translate meters to points squared
# Assuming figure size is (10, 10) inches and plot extent is 40 meters
fig_size_inch = 10
extent_meters = big_map_extent[3] - big_map_extent[2]
meters_to_points = fig_size_inch * 56 / extent_meters  # 72 points per inch #but 31 works here, 63 worked above...

# Calculate the areas in points squared
sizes = [np.pi * (radius * meters_to_points) ** 2 for radius in radii]

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

for i in range(len(x)):
    circle = Circle((x[i], y[i]), radii[i], alpha=0.6, edgecolor='b', facecolor='b')
    ax.add_patch(circle)
    ax.text(x[i], y[i], f'L={round(radii[i]/1000)} km', fontsize=12, ha='right')

# # Create scatter plot
ax.scatter(x, y, s=sizes, 
           marker='s', alpha=0.6, fc='r', ec='r')

# Show grid
ax.grid(True)

# lat_lon_tick_labels([-122, -120, -118, -116, -114], [32, 34, 36, 38], ax=ax)

big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
ax.axis(big_map_extent)

source = cx.providers.Esri.WorldTerrain
cx.add_basemap(ax, source=source)

plt.show()


### 2.1.1 Size of a single plot

In [ ]:
catalog_name = 'SCSN'
mainshocks = mainshock_dict[catalog_name].copy()

with plt.rc_context(rc_params_dict):
    fig = plt.figure(figsize=(8,8))
    i=0
    ax = fig.add_subplot(111)
    labels = [letter + ')' for letter in alphabet[:4]]
    ax.set_title(labels[i], loc='left')
    fault_gdf.plot(color='black', linewidth=0.5, ax=ax, zorder=20)

    i=0
    for key, item in catalogue_dict.items():
        plot_params = cat_options_dict[key]
        item = item.sample(n=100000).copy()
        item.sort_values(by='MAGNITUDE', inplace=True, ascending=True)
        ax.scatter(item['geometry'].x, item['geometry'].y, 
                    color=plot_color_dict[plot_params['color']], zorder=plot_params['zorder'], s=plot_params['size'](item['MAGNITUDE']),
                    ec='white', linewidth=0.25)
        i+=1

    # # Plotting mainshocks
    # ax.annotate('SCSN', EPSG_transformer((-114, 37))[0], fontsize=20, zorder=5, bbox=bbox_args, ha='right')
    # plot_options_dict = {'Both':{'zorder':11, 'color':'teal', 'size':(lambda x: 6*x**2)},
    #                     'FET':{'zorder':12, 'color':'orange', 'size':(lambda x: 6*x**2)},
    #                     'MDET':{'zorder':13, 'color':'purple', 'size':(lambda x: 6*x**2)},
    #                     'Neither':{'zorder':14, 'color':'grey', 'size':(lambda x: 10)}
    #                     }

    # legend_labels = []
    # c = 0
    # for selection, item in plot_options_dict.items():
    #     mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()
        
    #     ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=0.95, s=item['size'](mainshock_selection['MAGNITUDE']),
    #                 # color=colour_dict[selection_colours[selection]], 
    #                 color=plot_color_dict[item['color']],
    #                 zorder=item['zorder'], ec='white', linewidth=0.5,
    #                 label=f"{selection}: {len(mainshock_selection)}")
    #     legend_labels.append(f"{selection}: {len(mainshock_selection)}")
    #     c+=1

    # legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=10, label=legend_labels[0], linestyle='None'),
    #                 Line2D([], [], color=plot_colors[1], marker='o', markersize=10, label=legend_labels[1], linestyle='None'),
    #                 Line2D([], [], color=plot_colors[2], marker='o', markersize=10, label=legend_labels[2], linestyle='None'),
    #                 Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=5, label=legend_labels[3], linestyle='None')
    #                 ]

    # ax.legend(handles=legend_handles, loc='lower left', framealpha=1, bbox_to_anchor=(0, 0.02), ncol=1, fontsize=11)

    source = cx.providers.Esri.WorldTerrain
    cx.add_basemap(ax, source=source)
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15)
    lat_lon_tick_labels([-122, -120, -118, -116, -114], [32, 34, 36, 38], ax=ax)
    plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['green'], zorder=16)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)

    plt.savefig('../outputs/rough/map_test.png')

### Size of a four panel plot

In [ ]:
catalog_name = 'SCSN'
mainshocks = mainshock_dict[catalog_name].copy()
bbox_args = dict(boxstyle="round,pad=0.2", fc='white', alpha=0.8)
labels = [letter + ')' for letter in alphabet[:4]]

with plt.rc_context(rc_params_dict):

    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)

    for i in range(0,4):
        ax = fig.add_subplot(2,2,i+1)
        ax.set_title(labels[i], loc='left')
        ax.annotate('SCSN', EPSG_transformer((-113.9, 37))[0], fontsize=size_dict['textsize']*scale/3, zorder=5, bbox=bbox_args, ha='right')

        fault_gdf.plot(color='black', linewidth=1, ax=ax)

        legend_labels = []
        c = 0
        for selection, item in plot_options_dict.items():
            mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()
            
            ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=0.95, s=item['size'](mainshock_selection['MAGNITUDE'])*scale,
                        color=plot_color_dict[item['color']],
                        zorder=item['zorder'], ec='white', linewidth=item['linewidth'])
            if selection in ['FET', 'MDET']:
                selection = f"{selection} only"
            legend_labels.append(f"{selection}: {len(mainshock_selection)}")
            c+=1

        legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None', linewidth=0.5),
                        Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[1]}", linestyle='None', linewidth=0.5),
                        Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[2]}", linestyle='None', linewidth=0.5),
                        Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['neither_markersize'], label=legend_labels[3], linestyle='None', linewidth=0.25)
                        ]

        ax.legend(handles=legend_handles, loc='lower left', framealpha=0.5, bbox_to_anchor=(-0.01, 0.01), ncol=1,
                   handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)
        

        source = cx.providers.Esri.WorldTerrain
        cx.add_basemap(ax, source=source)
        add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'])
        lat_lon_tick_labels([-122, -118, -114], [32, 34, 36, 38], fontsize=size_dict['textsize'], 
                            ax=ax)
        plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['teal'], zorder=10, linewidth=size_dict['linewidth'])
        big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
        ax.axis(big_map_extent)

    fig.subplots_adjust(wspace=0.3, hspace=0)
    plt.show()

### Creating my own matplotlib style

In [ ]:
plt.style.use('four_panel_plot.mplstyle')
fig = plt.figure()

for i in range(0,4):
    ax = fig.add_subplot(2,2,i+1)
    ax.set_title(labels[i])
    ax.annotate('SCSN', EPSG_transformer((-113.9, 37))[0], fontsize=size_dict['textsize']*scale/3,
                 zorder=5, bbox=bbox_args, ha='right')

    fault_gdf.plot(color='black', linewidth=1, ax=ax)

    legend_labels = []
    c = 0
    for selection, item in plot_options_dict.items():
        mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()
        
        ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=0.95, s=item['size'](mainshock_selection['MAGNITUDE'])*scale,
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec='white', linewidth=item['linewidth'])
        if selection in ['FET', 'MDET']:
            selection = f"{selection} only"
        legend_labels.append(f"{selection}: {len(mainshock_selection)}")
        c+=1

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[1]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[2]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['neither_markersize'], label=legend_labels[3], linestyle='None', linewidth=0.25)
                    ]

    ax.legend(handles=legend_handles, loc='lower left', framealpha=0.5, bbox_to_anchor=(-0.01, 0.01), ncol=1,
                handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)
    

    source = cx.providers.Esri.WorldTerrain
    cx.add_basemap(ax, source=source)
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'])
    lat_lon_tick_labels([-122, -118, -114], [32, 34, 36, 38], fontsize=size_dict['textsize'], 
                        ax=ax)
    plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['teal'], zorder=10, linewidth=size_dict['linewidth'])
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)

fig.subplots_adjust(wspace=0.3, hspace=0.3)
plt.show()

In [ ]:
plt.style.use('one_panel_plot.mplstyle')
catalog_name = 'SCSN'
mainshocks = mainshock_dict[catalog_name].copy()

fig = plt.figure()
i=0
ax = fig.add_subplot(111)
labels = [letter + ')' for letter in alphabet[:4]]
ax.set_title(labels[i], loc='left')
bbox_args = dict(boxstyle="round,pad=0.2", fc='white', alpha=0.8)
# USGS_gdf.plot(color='black', linewidth=1, ax=ax)
fault_gdf.plot(color='black', linewidth=0.5, ax=ax, zorder=20)

# Plotting seismicity
size_relation = (lambda x: np.exp(1.1*x))
# size_relation = (lambda x: 5**(x)/10)
# size_relation = (lambda x: Leonard_rupture_length(x))
cat_options_dict = {'SCSN':{'zorder':13, 'color':'teal', 'size':size_relation},
                    'QTM_12':{'zorder':12, 'color':'orange', 'size':size_relation},
                    'QTM_9_5':{'zorder':11, 'color':'purple', 'size':size_relation},
                    }
i=0
for key, item in catalogue_dict.items():
    plot_params = cat_options_dict[key]
    item = item.sample(n=100000).copy()
    item.sort_values(by='MAGNITUDE', inplace=True, ascending=True)
    ax.scatter(item['geometry'].x, item['geometry'].y, 
                color=plot_color_dict[plot_params['color']], zorder=plot_params['zorder'], s=plot_params['size'](item['MAGNITUDE']),
                ec='white', linewidth=0.25)
    i+=1

# # Plotting mainshocks
# ax.annotate('SCSN', EPSG_transformer((-114, 37))[0], fontsize=20, zorder=5, bbox=bbox_args, ha='right')
# plot_options_dict = {'Both':{'zorder':11, 'color':'teal', 'size':(lambda x: 6*x**2)},
#                     'FET':{'zorder':12, 'color':'orange', 'size':(lambda x: 6*x**2)},
#                     'MDET':{'zorder':13, 'color':'purple', 'size':(lambda x: 6*x**2)},
#                     'Neither':{'zorder':14, 'color':'grey', 'size':(lambda x: 10)}
#                     }

# legend_labels = []
# c = 0
# for selection, item in plot_options_dict.items():
#     mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()
    
#     ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=0.95, s=item['size'](mainshock_selection['MAGNITUDE']),
#                 # color=colour_dict[selection_colours[selection]], 
#                 color=plot_color_dict[item['color']],
#                 zorder=item['zorder'], ec='white', linewidth=0.5,
#                 label=f"{selection}: {len(mainshock_selection)}")
#     legend_labels.append(f"{selection}: {len(mainshock_selection)}")
#     c+=1

# legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=10, label=legend_labels[0], linestyle='None'),
#                 Line2D([], [], color=plot_colors[1], marker='o', markersize=10, label=legend_labels[1], linestyle='None'),
#                 Line2D([], [], color=plot_colors[2], marker='o', markersize=10, label=legend_labels[2], linestyle='None'),
#                 Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=5, label=legend_labels[3], linestyle='None')
#                 ]

# ax.legend(handles=legend_handles, loc='lower left', framealpha=1, bbox_to_anchor=(0, 0.02), ncol=1, fontsize=11)


source = cx.providers.Esri.WorldTerrain
cx.add_basemap(ax, source=source)
add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15)
lat_lon_tick_labels([-122, -120, -118, -116, -114], [32, 34, 36, 38], ax=ax)
plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['teal'], zorder=16)
big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
ax.axis(big_map_extent)
plt.show()

## 2.2 The finished product

In [ ]:
event_bbox_args = dict(boxstyle="round,pad=0.2", fc='white', alpha=0.75, ec='red')
names = ['San Andreas fault zone',
            'San Jacinto fault',
            'Owens Valley fault zone',
            'Elsinore fault zone',
            'Garlock fault zone',
            'Calaveras fault zone',
            'Brawley Seismic Zone']

bbox_args = dict(boxstyle="round,pad=0.2", fc='white', alpha=0.8)
fault_bbox_args = dict(boxstyle="round,pad=0.2", fc='white', alpha=0.75)

fig = plt.figure(figsize=(8,6), dpi=300)
ax = fig.add_subplot(111)
ax = fault_gdf.plot(color='black', linewidth=1, ax=ax)
# ax = gdf_points.plot(ax=ax, markersize=0.5, alpha=0.1, color=colour_dict['green'])
i=0
for key, item in geometry_dict.items():
    item = item.to_crs(epsg=3857)
    # item = item.iloc[0:100000].copy()
    # ax = item.plot(ax=ax, markersize=0.5, alpha=0.1, color=colours[i])
    i+=1

plotted_names = set()
name_plot_file = fault_gdf.loc[fault_gdf['Name'].isin(names)] 
for idx, row in name_plot_file.iterrows():
    name = row['Name']
    if name not in plotted_names:
        ax.annotate(name, (row['geometry'].centroid.x+0.15, row['geometry'].centroid.y), fontsize=6, zorder=2, bbox=fault_bbox_args)
        plotted_names.add(name)

largest_events_gdf.plot(color='red', alpha=0.5, marker='*', ax=ax, markersize=2)

for event in largest_events_gdf.itertuples():
    x, y = event.geometry.x, event.geometry.y
    focmecs = [event.strike, event.dip, event.rake]
    print(x, y, focmecs)
    
    b = beach(focmecs, xy=(x, y), width=50000, linewidth=1, alpha=0.85, facecolor='r')
    ax.annotate(event.NAME, (x-80000, y+30000), fontsize=6, zorder=2, bbox=event_bbox_args)

    b.set_zorder(1)
    ax.add_collection(b)

source = cx.providers.Esri.WorldTerrain
cx.add_basemap(ax, source=source)
lat_lon_tick_labels([-122, -120, -118, -116, -114], [32, 34, 36, 38], ax=ax)
# add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax)

plt.savefig('../outputs/rough/fault_maps_with_terrain_and_beachballs.png')

# 3. Catalogue figure
Add names of faults? Unsure, would need to resize and maybe cut some.

In [ ]:
catalogue_dict = {key: catalogue_dict[key] for key in catalogue_names}

scale = 2 #3
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':0.5,
             'legend_markersize':15*scale,
             'point_alpha':0.6
             }

size_relation = (lambda x: 10 + np.exp(1.1*x))
cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7, 'ref':'Hutton et al., 2010'},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019'},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019'}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/1.5}):

    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,:])
    ax2 = fig.add_subplot(gs[1,0])
    ax3 = fig.add_subplot(gs[1,1])

    # ax1 = fig.add_subplot(211)
    ax1.set_title(f"a)", loc='left')
    ax1.set_xlabel(f"Year")
    ax1.set_ylabel(f"Magnitude")

    # ax = fig.add_subplot(223)
    ax2.set_title(f"b)", loc='left')
    fault_gdf.plot(color='black', linewidth=1, ax=ax2, zorder=200)

    # ax3 = fig.add_subplot(224)
    ax3.set_title(f"c)", loc='left')
    ax3.set_xlabel('Magnitude')
    ax3.set_ylabel('Frequency')

    i = 0
    for name, catalogue in catalogue_dict.items():
        plot_params = cat_options_dict[name]
        # catalogue = catalogue.iloc[:10000].copy()
        z = np.exp(catalogue['MAGNITUDE'])/5
        ax1.scatter(catalogue['DATETIME'], catalogue['MAGNITUDE'], alpha=size_dict['point_alpha'], color=plot_params['color'], ec='white',
                    s=plot_params['size'](catalogue['MAGNITUDE'])*scale, linewidth=size_dict['edge_width']/2, zorder=plot_params['zorder'])

        catalogue.sort_values(by='MAGNITUDE', inplace=True, ascending=True)
        ax2.scatter(catalogue['geometry'].x, catalogue['geometry'].y, alpha=size_dict['point_alpha'],
                    color=plot_params['color'], zorder=plot_params['zorder'], s=plot_params['size'](catalogue['MAGNITUDE'])*scale,
                        ec='white', linewidth=size_dict['edge_width'])

        a = np.log10(len(catalogue))
        bins = np.array(range(math.floor(catalogue['MAGNITUDE'].min())*10, math.ceil(catalogue['MAGNITUDE'].max())*10,1))/10
        values, base = np.histogram(catalogue['MAGNITUDE'], bins=bins)
        cumulative = np.cumsum(values)
        mark = seispy.find_nearest(base, plot_params['Mc'])
        mark_index = np.where(base == mark)

        ax3.axvline(plot_params['Mc'], linewidth=size_dict['linewidth'],
                    color=plot_params['color'], 
                    alpha=0.9, linestyle='--',zorder=0)
        
        ax3.plot(base[:-1], len(catalogue)-cumulative,
                color=plot_params['color'], linewidth=size_dict['linewidth'],
                alpha=0.6
                )

        ax3.set_yscale("log")
        ax3.scatter(plot_params['Mc'], len(catalogue)-cumulative[mark_index], 
                    color=plot_params['color'],
                    label=fr"Mc: {plot_params['Mc']}",
                    marker="D", zorder=5, s=size_dict['legend_markersize']*scale*scale, 
                    )
        i+=1

    # ax2.scatter(El_Mayor_Cucupah['geometry'].x, El_Mayor_Cucupah['geometry'].y, ec='red', fc='none', zorder=100,
    #             s=500*scale,#s=1*np.exp(El_Mayor_Cucupah['MAGNITUDE']),
    #             marker='*', alpha=0.9)

    source = cx.providers.Esri.WorldTerrain
    cx.add_basemap(ax2, source=source)
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax2, color=plot_color_dict['brown'], zorder=15, linewidth=size_dict['linewidth'], alpha=0.9)
    lat_lon_tick_labels([-122, -118, -114], [32, 34, 36, 38], fontsize=size_dict['textsize'], ax=ax2)
    plot_SCSN_reporting_region(ax=ax2, color=plot_color_dict['green'], zorder=10, linewidth=size_dict['linewidth'], alpha=0.9)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax2.axis(big_map_extent)

    ax3.legend(loc='upper right')
    ax3.set_xticks(np.arange(-2,9,2))
    # ax3.set_yticks([0, 100, 10000, 1000000])

    panel_a_legend = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label='SCSN', linestyle='None'),
                      Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label='QTM 12', linestyle='None'),
                      Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label='QTM 9.5', linestyle='None')
                      ]

    ax1.legend(handles=panel_a_legend, loc='upper center', bbox_to_anchor=(0.68, 1.03), ncol=3)
    ax1.set_yticks([-2, 0, 2, 4, 6])
    ax1.set_yticklabels([-2, 0, 2, 4, 6])

    ax1.axhline(4, color='grey', linestyle='--', alpha=0.8, zorder=2000)
    rectangle_coords = [dt.datetime(2008,1,1), dt.datetime(2009,1,1), -3, 8]
    rectangle = Rectangle(
        (rectangle_coords[0], rectangle_coords[2]),
        rectangle_coords[1] - rectangle_coords[0],
        rectangle_coords[3] - rectangle_coords[2],
        linewidth=1, ec='grey', fc='grey', alpha=0.2, zorder=0
        )
    ax1.add_patch(rectangle)

    plt.tight_layout()
    plt.savefig('../outputs/figures/catalog_multiplot.png')
    fig.subplots_adjust(wspace=0.5, hspace=0.35)
    plt.show()

# 4. Map of candidate mainshocks by catalog

In [16]:
# mainshock_dict = {}
# for name, data in catalogue_dict.items():
#     mainshock_file = pd.read_csv(f'../data/{name}/mainshocks.csv')
#     string_to_datetime_df(mainshock_file)
#     mainshock_file['YEAR'] = seispy.datetime_to_decimal_year(mainshock_file['DATETIME'])
#     # mainshock_file.to_csv(f'../data/{name}/mainshocks.csv', index=False)
#     mainshock_file = df_to_gdf(mainshock_file)
#     mainshock_dict[name] = mainshock_file

# QTM_9_5_mainshocks = mainshock_dict['QTM_9_5'].copy()
# QTM_12_mainshocks = mainshock_dict['QTM_12'].copy()
# SCSN_mainshocks = mainshock_dict['SCSN'].copy()

# mainshocks_in_all_QTM_9_5 = QTM_9_5_mainshocks.loc[(QTM_9_5_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
#                                                         (QTM_9_5_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

# mainshocks_in_all_QTM_12 = QTM_12_mainshocks.loc[(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
#                                                         (QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID']))].copy()

# mainshocks_in_all_SCSN = SCSN_mainshocks.loc[(SCSN_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
#                                                         (SCSN_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

# mainshocks_in_all_dict = {'QTM_9_5':mainshocks_in_all_QTM_9_5,
#                           'QTM_12':mainshocks_in_all_QTM_12,
#                           'SCSN':mainshocks_in_all_SCSN}

# QTM_9_5_only_mainshocks = QTM_9_5_mainshocks.loc[~(QTM_9_5_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
#                                                  ~(QTM_9_5_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

# QTM_12_only_mainshocks = QTM_12_mainshocks.loc[~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
#                                                ~(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID']))].copy  ()

# SCSN_only_mainshocks = SCSN_mainshocks.loc[~(SCSN_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
#                                            ~(SCSN_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

# QTM_only_mainshocks =  QTM_12_mainshocks.loc[(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
#                                              ~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID']))].copy()

In [ ]:
textsize = 40
edge_width = 0.5
box_width = 3
grey_alpha = 0.3
plt.rcParams['lines.linewidth'] = edge_width
with plt.rc_context({'axes.titlesize': textsize,
                     'axes.labelsize': textsize, 
                     'xtick.labelsize':textsize,
                       'ytick.labelsize':textsize,
                       'lines.markersize':30,
                       'legend.fontsize':textsize/1.5}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,:])
    ax2 = fig.add_subplot(gs[1,0])
    ax3 = fig.add_subplot(gs[1,1])

    # ax1.scatter(mainshocks_in_all_QTM_12['YEAR'], mainshocks_in_all_QTM_12['MAGNITUDE'], fc='black', ec='white', 
    #             label=f'All catalogs (QTM data): {len(mainshocks_in_all_QTM_12)}', linewidths=edge_width, alpha=grey_alpha)
    ax1.scatter(mainshocks_in_all_SCSN['YEAR'], mainshocks_in_all_SCSN['MAGNITUDE'], fc='grey', ec='white', 
                label=f'In all catalogs: {len(mainshocks_in_all_SCSN)}', linewidths=edge_width, alpha=grey_alpha)
    ax1.scatter(SCSN_only_mainshocks['YEAR'], SCSN_only_mainshocks['MAGNITUDE'], fc=plot_color_dict['teal'], 
                ec='white', label=f'In SCSN only: {len(SCSN_only_mainshocks)}', linewidths=edge_width)
    ax1.scatter(QTM_only_mainshocks['YEAR'], QTM_only_mainshocks['MAGNITUDE'], fc=plot_color_dict['orange'], 
                ec='white', label=f'In both QTM only: {len(QTM_only_mainshocks)}', linewidths=edge_width)
    ax1.scatter(QTM_9_5_only_mainshocks['YEAR'], QTM_9_5_only_mainshocks['MAGNITUDE'], fc=plot_color_dict['purple'], ec='white',
                 label=f'In QTM 9.5 only: {len(QTM_9_5_only_mainshocks)}', linewidths=edge_width)
    ax1.legend(loc='upper right', bbox_to_anchor=(1.01, 1.025))
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Magnitude')
    ax1.set_title('a)', loc='left')
    ax1.set_xticks(np.arange(2008, 2020, 2))
    ax1.set_xticklabels(np.arange(2008, 2020, 2))

    # ax2.scatter(mainshocks_in_all_QTM_12['geometry'].x, mainshocks_in_all_QTM_12['geometry'].y, color='black', ec='white', linewidths=edge_width, alpha=grey_alpha)
    ax2.scatter(mainshocks_in_all_SCSN['geometry'].x, mainshocks_in_all_SCSN['geometry'].y, color='grey', ec='white', linewidths=edge_width, alpha=grey_alpha)
    ax2.scatter(SCSN_only_mainshocks['geometry'].x, SCSN_only_mainshocks['geometry'].y, fc=plot_color_dict['teal'], ec='white', linewidths=edge_width)
    ax2.scatter(QTM_9_5_only_mainshocks['geometry'].x, QTM_9_5_only_mainshocks['geometry'].y, fc=plot_color_dict['purple'], ec='white', linewidths=edge_width)
    ax2.scatter(QTM_only_mainshocks['geometry'].x, QTM_only_mainshocks['geometry'].y, fc=plot_color_dict['orange'], ec='white', linewidths=edge_width)
    ax2.set_title('b)', loc='left')

    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax2.axis(big_map_extent)
    source = cx.providers.Esri.WorldTerrain
    cx.add_basemap(ax2, source=source)
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax2, color=plot_color_dict['brown'], zorder=0, linewidth=box_width, alpha=0.9)
    zoom_extent = [-116,-114.8, 31.8, 33] 
    add_box_3857(zoom_extent, ax=ax2, zorder=100, linewidth=box_width, alpha=0.9, color=plot_color_dict['pink'])
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], fontsize=textsize, ax=ax2)
    plot_SCSN_reporting_region(ax=ax2, color=plot_color_dict['green'], zorder=0, linewidth=box_width, alpha=0.9)
    fault_gdf.plot(color='black', linewidth=0.5, ax=ax2, zorder=20)

    # ax3.scatter(mainshocks_in_all_QTM_12['geometry'].x, mainshocks_in_all_QTM_12['geometry'].y, color='black', ec='white', linewidths=edge_width, alpha=grey_alpha)
    ax3.scatter(mainshocks_in_all_SCSN['geometry'].x, mainshocks_in_all_SCSN['geometry'].y, color='grey', ec='white', linewidths=edge_width, alpha=grey_alpha)
    ax3.scatter(SCSN_only_mainshocks['geometry'].x, SCSN_only_mainshocks['geometry'].y, fc=plot_color_dict['teal'], ec='white', linewidths=edge_width)
    ax3.scatter(QTM_9_5_only_mainshocks['geometry'].x, QTM_9_5_only_mainshocks['geometry'].y, fc=plot_color_dict['purple'], ec='white', linewidths=edge_width)
    ax3.scatter(QTM_only_mainshocks['geometry'].x, QTM_only_mainshocks['geometry'].y, fc=plot_color_dict['orange'], ec='white', linewidths=edge_width)
    ax3.set_title('c)', loc='left')

    extent = convert_extent_to_epsg3857([-116.5, -114.5, 31.5, 33.5])
    ax3.axis(extent)
    source = cx.providers.Esri.WorldTerrain
    cx.add_basemap(ax3, source=source)
    add_box_3857(zoom_extent, ax=ax3, zorder=100, linewidth=box_width, alpha=0.9, color=plot_color_dict['pink'])
    lat_lon_tick_labels([-116, -115], [32, 33], ax=ax3, fontsize=textsize)
    fault_gdf.plot(color='black', linewidth=0.5, ax=ax3, zorder=20)
    fig.subplots_adjust(hspace=-0.5)
    # pos = ax2.get_position()
    # new_pos = [pos.x0 - 0.7, pos.y0, pos.width, pos.height]
    # ax2.set_position(new_pos)
    plt.tight_layout()
    plt.savefig('../outputs/figures/candidate_mainshocks.png')
    plt.show()

In [ ]:
pd.merge(QTM_only_mainshocks[['ID', 'MAGNITUDE']], SCSN[['ID', 'MAGNITUDE']], how='inner', on='ID')

# 5. Map of mainshock selection using SCSN and QTM locations

In [ ]:
merged_mainshocks = pd.merge(mainshocks_in_all_SCSN, mainshocks_in_all_QTM_12, on='ID', suffixes=('_SCSN', '_QTM_12'))
differing_selections = merged_mainshocks.loc[merged_mainshocks['Selection_QTM_12']!=merged_mainshocks['Selection_SCSN']].copy()
print(len(differing_selections))
differing_selections[['ID', 'Selection_SCSN', 'Selection_QTM_12', 'DATETIME_QTM_12', 'TR_excluded_by_SCSN', 'TR_excluded_by_QTM_12', 'LON_SCSN', 'LAT_SCSN', 'geometry_SCSN']].sort_values(by=['Selection_SCSN', 'Selection_QTM_12'])

In [ ]:
bbox_args = dict(boxstyle="round,pad=0.05", fc='white', alpha=0.8)

scale = 2 #3
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

size_relation = (lambda x: (200*scale + np.exp(1.1*x))*scale)
cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7, 
                            'ref':'Hutton et al., 2010', 'label': 'SCSN', 'fill':'left', 'bbox':(1, 1)},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3,
                               'ref':'Ross et al., 2019', 'label': 'QTM 12', 'fill':'right', 'bbox':(1, 0.5)},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3,
                                'ref':'Ross et al., 2019', 'label': 'QTM 9.5'},
                    'Combined':{'label': 'Merged'}
                    }

plot_options_dict = {'Both':{'zorder':11, 'color':'teal', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'FET':{'zorder':12, 'color':'orange', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'MDET':{'zorder':13, 'color':'purple', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'Neither':{'zorder':10, 'color':'yellow', 'size':size_relation, 'linewidth':0.5, 'ec':'white'}
                    }

selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':15, 'color':plot_color_dict['yellow'], 'size':(lambda x: (5*scale + np.exp(1.1*x))/scale), 'linewidth':0.5}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):

  fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)

  i = 0
  labels = [letter + ')' for letter in alphabet[:6]]
  mainshocks_in_all_dict = {'SCSN':mainshocks_in_all_SCSN,
                            'QTM_12':mainshocks_in_all_QTM_12}
  difference_region = [-116.5, -114.5, 31.5, 33]
  text_x, text_y = -114.55, 32.85
  ax = fig.add_subplot(2,1,1)
  ax.set_title('a)', loc='left')
  ax2 = fig.add_subplot(2,1,2)
  ax2.set_title('b)', loc='left')
  legend_labels = []
  test = []
  for cat, main_cat in mainshocks_in_all_dict.items():
    cat_params = cat_options_dict[cat]
    print(cat, cat_params['fill'])
    main_cat = seispy.restrict_catalogue_geographically(main_cat, region=difference_region)
    main_cat = main_cat.loc[(main_cat['DATETIME'] > dt.datetime(2008,1,1)) & (main_cat['DATETIME'] < dt.datetime(2012,6,1))]

    ax.scatter(main_cat["geometry"].x, main_cat["geometry"].y,
                color='grey', zorder=10, alpha=0.5, marker=MarkerStyle("o", fillstyle=cat_params['fill']),
                s=cat_params['size'](main_cat['MAGNITUDE']), ec='white', linewidth=0.25)

    years = seispy.datetime_to_decimal_year(main_cat['DATETIME'])
    ax2.scatter(years, main_cat['MAGNITUDE'], s=cat_params['size'](main_cat['MAGNITUDE']), alpha=0.5,
                color='grey', ec='white', linewidth=0.25)
    ax2.set_xlim(2008, 2012.5)

    c = 0
    for selection, item in plot_options_dict.items():
        mainshock_selection = differing_selections.loc[differing_selections[f'Selection_{cat}']==selection].copy()
        ax.scatter(mainshock_selection[f'geometry_{cat}'].x, mainshock_selection[f'geometry_{cat}'].y, 
                    alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}']),
                    color=plot_color_dict[item['color']], marker=MarkerStyle("o", fillstyle=cat_params['fill']),
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        years = seispy.datetime_to_decimal_year(mainshock_selection[f"DATETIME_{cat}"])
        ax2.scatter(years, mainshock_selection[f'MAGNITUDE_{cat}'],
                    alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}']),
                    color=plot_color_dict[item['color']], marker=MarkerStyle("o", fillstyle=cat_params['fill']),
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        if selection in ['FET', 'MDET']:
            selection = f"{selection} only"
        elif selection=='Both':
            selection='DDET'
        legend_labels.append(f"{selection}: {len(mainshock_selection)}")
        c+=1
    
    source = cx.providers.Esri.WorldTerrain
    lat_lon_tick_labels([-117, -116, -115, -114], [32, 33], ax=ax, fontsize=size_dict['textsize'])
    extent = convert_extent_to_epsg3857([-117.5, -114.5, 31.7, 33.1])
    ax.axis(extent)
    cx.add_basemap(ax, source=source)
    add_box_3857(zoom_extent, ax=ax, zorder=10, linewidth=3, alpha=0.9, color=plot_color_dict['pink'])
    fault_gdf.plot(color='black', linewidth=0.5, ax=ax, zorder=20)

    for (x1_pt, y1_pt, x2_pt, y2_pt) in zip(differing_selections['geometry_SCSN'].x, differing_selections['geometry_SCSN'].y, differing_selections['geometry_QTM_12'].x, differing_selections['geometry_QTM_12'].y):
       ax.plot([x1_pt, x2_pt], [y1_pt, y2_pt], color='black', linestyle='-', linewidth=2.5, alpha=0.5, zorder=100)


  l = MarkerStyle("o", fillstyle='left')
  r = MarkerStyle("o", fillstyle='right')

  legend_properties ={'xdata':[],
                      'ydata':[],
                      'markersize':size_dict['legend_markersize'],
                      'linestyle':'None', 
                      'linewidth':0.5,
                      'markeredgecolor':'white'
                      }
  legend_handles = [#Line2D([], [], color='None', marker='None', label=''),
                    Line2D([], [], color='None', marker='None', label='SCSN:'),
                    Line2D(**legend_properties, color=plot_colors[0], marker=l, label=legend_labels[0]),
                Line2D(**legend_properties, color=plot_colors[1], marker=l, label=f"{legend_labels[1]}"),
                Line2D(**legend_properties, color=plot_colors[2], marker=l, label=f"{legend_labels[2]}"),
                Line2D(**legend_properties, color=plot_color_dict['yellow'], marker=l, label=legend_labels[3]),
                Line2D([], [], color='None', marker='None', label=''),
                Line2D([], [], color='None', marker='None', label='QTM 12:'),
                Line2D(**legend_properties, color=plot_colors[0], marker=r, label=legend_labels[4]),
                Line2D(**legend_properties, color=plot_colors[1], marker=r, label=f"{legend_labels[5]}"),
                Line2D(**legend_properties, color=plot_colors[2], marker=r, label=f"{legend_labels[6]}"),
                Line2D(**legend_properties, color=plot_color_dict['yellow'], marker=r, label=legend_labels[7])
                ]

  ax.legend(handles=legend_handles, loc='upper left', framealpha=1, bbox_to_anchor=(0,1), ncol=1, fontsize=size_dict['textsize']/2,
            handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)
  
  # ax.annotate('SCSN:', (-13050017, 3891486), size=20, zorder=2000)
  # ax.annotate('QTM 12:', (-13052017, 3810000), size=20, zorder=2000)

  pos = ax.get_position()
  new_pos = [pos.x0 - 0.45, pos.y0, pos.width, pos.height]
  ax.set_position(new_pos)

  ax2.set_ylabel('Magnitude')
  ax2.set_xlabel('Year')

  plt.tight_layout()
  # fig.subplots_adjust(wspace=0.2, hspace=0.5)

  plt.savefig(f"../outputs/figures/changing_selections.png", bbox_inches='tight')
  plt.show()

## Old

In [ ]:
bbox_args = dict(boxstyle="round,pad=0.05", fc='white', alpha=0.8)

scale = 2 #3
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)
cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7, 'ref':'Hutton et al., 2010', 'label': 'SCSN'},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 12'},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 9.5'},
                    'Combined':{'label': 'Merged'}
                    }

plot_options_dict = {'Both':{'zorder':11, 'color':'teal', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'FET':{'zorder':12, 'color':'orange', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'MDET':{'zorder':13, 'color':'purple', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'Neither':{'zorder':10, 'color':'yellow', 'size':size_relation, 'linewidth':0.5, 'ec':'white'}
                    }

selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':15, 'color':plot_color_dict['yellow'], 'size':(lambda x: (5*scale + np.exp(1.1*x))/scale), 'linewidth':0.5}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):

  fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)

  i = 0
  labels = [letter + ')' for letter in alphabet[:6]]
  mainshocks_in_all_dict = {'SCSN':mainshocks_in_all_SCSN,
                            'QTM_12':mainshocks_in_all_QTM_12}
  for name, mainshocks in mainshocks_in_all_dict.items():
    cat_params = cat_options_dict[name]
    ax = fig.add_subplot(3,2,i+1)
    ax.set_title(labels[i], loc='left')
    # ax.annotate(cat_params['label'], EPSG_transformer(size_dict['label_pos'])[0], fontsize=size_dict['textsize'], zorder=50, bbox=bbox_args, ha='left')

    legend_labels = []
    for selection, params in selection_dict.items():
      mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()      
      ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=size_dict['point_alpha'],
                  s=params['size'](mainshock_selection['MAGNITUDE']), color=params['color'],
                  zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                  label=f"{selection}: {len(mainshock_selection)}")
      # legend_labels.append(f"{selection}: {len(mainshock_selection)}")
      if selection in ['FET', 'MDET']:
        selection = f"{selection} only"
      elif selection=='Both':
        selection='DDET'
      legend_labels.append(f"{selection}: {len(mainshock_selection)}")

    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax, zorder=1)
    source = cx.providers.Esri.WorldTerrain
    # add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'], alpha=0.9)
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], fontsize=size_dict['textsize'], ax=ax)
    zoom_extent = [-116,-114.8, 31.8, 33] 
    add_box_3857(zoom_extent, ax=ax, zorder=100, linewidth=3, alpha=0.9, color=plot_color_dict['pink'])
    # plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['green'], zorder=10, linewidth=size_dict['linewidth'], alpha=0.9)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)
    cx.add_basemap(ax, source=source)

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None'),
                      Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[1], linestyle='None'),
                      Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[2], linestyle='None'),
                      Line2D([], [], color=plot_color_dict['yellow'], marker='o', markersize=size_dict['legend_markersize']/2, label=legend_labels[3], linestyle='None')
                      ]
    
    # ax.legend(handles=legend_handles, loc='lower left', bbox_to_anchor=(0, 0), ncol=2, framealpha=0.7,
    #           handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    ax.legend(handles=legend_handles, title=name, loc='upper left', bbox_to_anchor=(1, 1.05), ncol=1, framealpha=0.7,
              handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    if i==1:
      ax.set_yticklabels([])
    i+=1

  difference_region = [-116.5, -114.5, 31.5, 33]
  text_x, text_y = -114.55, 32.85
  for cat, main_cat in mainshocks_in_all_dict.items():
    ax = fig.add_subplot(3,2,i+1)
    ax.set_title(labels[i], loc='left')
    # ax.annotate(cat, EPSG_transformer((text_x, text_y))[0], zorder=50, bbox=bbox_args, ha='right', fontsize=size_dict['textsize'])

    main_cat = seispy.restrict_catalogue_geographically(main_cat, region=difference_region)
    main_cat = main_cat.loc[(main_cat['DATETIME'] > dt.datetime(2008,1,1)) & (main_cat['DATETIME'] < dt.datetime(2012,6,1))]
    plot_params = cat_options_dict[cat]

    ax.scatter(main_cat["geometry"].x, main_cat["geometry"].y, 
                color='grey', zorder=10, alpha=0.5,
                s=plot_params['size'](main_cat['MAGNITUDE']), ec='white', linewidth=0.25)
    years = seispy.datetime_to_decimal_year(main_cat['DATETIME'])

    ax2 = fig.add_subplot(3,2,i+3)
    ax2.set_title(labels[i+2], loc='left')
    ax2.scatter(years, main_cat['MAGNITUDE'], s=plot_params['size'](main_cat['MAGNITUDE']), alpha=0.5,
                color='grey', ec='white', linewidth=0.25)
    ax2.set_xlim(2008, 2012.5)

    legend_labels = []
    c = 0
    for selection, item in plot_options_dict.items():
        mainshock_selection = differing_selections.loc[differing_selections[f'Selection_{cat}']==selection].copy()
        ax.scatter(mainshock_selection[f'geometry_{cat}'].x, mainshock_selection[f'geometry_{cat}'].y, 
                    alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}']),
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        years = seispy.datetime_to_decimal_year(mainshock_selection[f"DATETIME_{cat}"])
        ax2.scatter(years, mainshock_selection[f'MAGNITUDE_{cat}'],
                    alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}']),
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        if selection in ['FET', 'MDET']:
            selection = f"{selection} only"
        elif selection=='Both':
            selection='DDET'
        legend_labels.append(f"{selection}: {len(mainshock_selection)}")
        c+=1

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[1]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[2]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_color_dict['yellow'], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[3], linestyle='None', linewidth=0.5)
                    ]

    ax.legend(handles=legend_handles, title=cat, loc='upper left', framealpha=1, bbox_to_anchor=(1, 1.05), ncol=1, fontsize=size_dict['textsize']/2,
            handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)
    
    source = cx.providers.Esri.WorldTerrain
    lat_lon_tick_labels([-116, -115, -114], [32, 33], ax=ax, fontsize=size_dict['textsize'])
    extent = convert_extent_to_epsg3857([-116.5, -114.5, 31.5, 33])
    ax.axis(extent)
    cx.add_basemap(ax, source=source)
    add_box_3857(zoom_extent, ax=ax, zorder=10, linewidth=3, alpha=0.9, color=plot_color_dict['pink'])
    fault_gdf.plot(color='black', linewidth=0.5, ax=ax, zorder=20)

    i+=1

  # i=0
  # for axs in [ax, ax2, ax3, ax4]:        
  #     axs.set_title(labels[i], loc='left')
  #     i+=1

  # for axs in [ax2,ax4]:
  #     axs.set_ylabel('Magnitude')
  #     axs.set_xlabel('Year')

  plt.tight_layout()
  fig.subplots_adjust(wspace=0.2, hspace=0.5)

  all_axes = fig.get_axes()
  for i, ax in enumerate(all_axes):
    if i in [0,1,2,4]:
      pos = ax.get_position()
      new_pos = [pos.x0 - 0.07, pos.y0, pos.width, pos.height]
      ax.set_position(new_pos)
    if i in [3,5]:
      ax.set_xlabel('Year')
    if i ==3:
      ax.set_ylabel('Magnitude')
    if i in [4,5]:
      ax.set_yticklabels([])

  # plt.savefig(f"../outputs/figures/initial_mainshock_selection_map.png", bbox_inches='tight')
  plt.show()

# 6. Figure: Differences between FET and MDET mainshocks
Need to do something about panel b.  
Think I need to include events around the red box, but not show in time series, or count in legend.
If I want to go really savvy I could draw red lines to the excluding mainshock, just to make things really clear. Will do later if I have time.

In [ ]:
for selection in merged_mainshocks_catalog['Selection'].unique():
    print(f"{selection}: {len(merged_mainshocks_catalog.loc[merged_mainshocks_catalog['Selection']==selection])}")

In [ ]:
QTM_12_mainshocks_merged = QTM_12_mainshocks.loc[QTM_12_mainshocks['ID'].isin(merged_mainshocks_catalog['ID'])].copy()
for selection in QTM_12_mainshocks_merged['Selection'].unique():
    print(f"{selection}: {len(QTM_12_mainshocks_merged.loc[QTM_12_mainshocks_merged['Selection']==selection])}")

In [ ]:
scale=2
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

scale = 2
size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)
neither_relation = (lambda x: (5*scale + np.exp(1.1*x))/scale)
selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['yellow'], 'size':neither_relation, 'linewidth':0.25}
                    }

# plt.style.use('four_panel_plot.mplstyle')
labels = [letter + ')' for letter in alphabet[:4]]
# QTM_12_mainshocks = mainshock_dict['QTM_12'].copy()
El_Mayor = QTM_12_mainshocks_merged.loc[QTM_12_mainshocks_merged['MAGNITUDE']>7].iloc[0]

El_Mayor_data = QTM_12_mainshocks_merged.loc[(QTM_12_mainshocks_merged['DATETIME'] > pd.Timestamp(2010,1,1)) &\
                                      (QTM_12_mainshocks_merged['DATETIME'] < pd.Timestamp(2011,4,4))].copy()
El_Mayor_data_unrestricted = El_Mayor_data.copy()
print(len(El_Mayor_data_unrestricted))
# El_Mayor_data = seispy.restrict_catalogue_geographically(El_Mayor_data, region=[-116,-115, 32, 33])
El_Mayor_data = seispy.restrict_catalogue_geographically(El_Mayor_data, region=[-116.5, -114.5, 31.9, 33.1])

pre_El_Mayor = El_Mayor_data.loc[El_Mayor_data['DATETIME']<El_Mayor.DATETIME].copy()

# El_Mayor_data_dict = {'All':El_Mayor_data}
# QTM_12_mainshock_dict = {'All':mainshocks_in_all_SCSN}
# options = ['Neither', 'Both', 'MDET', 'FET']
# for option in options:
#     EM_data = El_Mayor_data.loc[El_Mayor_data['Selection']==option].copy()
#     QTM_12_data = QTM_12_mainshocks.loc[QTM_12_mainshocks['Selection']==option].copy()
#     print(f"{option}: {len(QTM_12_data)}")
#     El_Mayor_data_dict.update({option:EM_data})
#     QTM_12_mainshock_dict.update({option:QTM_12_data})

box_alpha = 0.2
with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(2,2)

    ax1 = fig.add_subplot(gs[0, 0])
    # ax1.set_extent([-121.953, -114.026, 31.86317, 37.22333], crs=ccrs.PlateCarree())
    ax2 = fig.add_subplot(gs[0, 1])
    # ax2.set_extent([-116,-115, 32, 33], crs=ccrs.PlateCarree())
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])

    i = 0
    QTM_legend_labels = []
    EM_legend_labels = []
    for option, params in selection_dict.items():
        EM_data = El_Mayor_data.loc[El_Mayor_data['Selection']==option].sort_values(by='MAGNITUDE', ascending=False)
        # EM_data_unrestricted = El_Mayor_data_unrestricted.loc[El_Mayor_data_unrestricted['Selection']==option].copy()
        # EM_data = EM_data_unrestricted.copy()
        EM_data_unrestricted = EM_data.copy()
        pre_EM = pre_El_Mayor.loc[pre_El_Mayor['Selection']==option].sort_values(by='MAGNITUDE', ascending=False)
        print(len(EM_data_unrestricted), option)
        mainshock_option_data = QTM_12_mainshocks_merged.loc[QTM_12_mainshocks_merged['Selection']==option]

        if option in ['FET', 'MDET']:
            option = f"{option}-only"
        if option == 'Both':
            option='DDET'
        if option=='Neither':
            option='None'
        QTM_legend_labels.append(f"{len(mainshock_option_data)} {option}")
        EM_legend_labels.append(f"{len(EM_data)} {option}")

        ax1.scatter(mainshock_option_data['geometry'].x, mainshock_option_data['geometry'].y,
            alpha=size_dict['point_alpha'], s=params['size'](mainshock_option_data['MAGNITUDE']), color=params['color'],
            zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
            label=f"{len(mainshock_option_data)} {option}")

        ax2.scatter(#EM_data['geometry'].x, EM_data['geometry'].y,
                    EM_data_unrestricted['geometry'].x, EM_data_unrestricted['geometry'].y, 
                    alpha=size_dict['point_alpha'], s=params['size'](EM_data_unrestricted['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(EM_data)} {option}")
        ax2.scatter(#EM_data['geometry'].x, EM_data['geometry'].y,
                    pre_EM['geometry'].x, pre_EM['geometry'].y, marker='s',
                    alpha=size_dict['point_alpha'], s=params['size'](pre_EM['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'])
        radius_km = 10
        new_LON, new_LAT = seispy.add_distance_to_position_pyproj(El_Mayor['LON'], El_Mayor['LAT'], radius_km, 0)
        radius_degrees = new_LON - El_Mayor['LON']
        circle_10km = Circle((El_Mayor['LON'], El_Mayor['LAT']), radius=radius_degrees, facecolor='none', edgecolor='red', alpha=0.2)
        # ax2.add_patch(circle_10km)
        QTM_years = seispy.datetime_to_decimal_year(mainshock_option_data['DATETIME'])
        ax3.scatter(QTM_years, mainshock_option_data['MAGNITUDE'],
                    alpha=size_dict['point_alpha'], s=params['size'](mainshock_option_data['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(mainshock_option_data)} {option}")
        EM_years = seispy.datetime_to_decimal_year(EM_data['DATETIME'])
        ax4.scatter(EM_years, EM_data['MAGNITUDE'],
                    alpha=size_dict['point_alpha'], s=params['size'](EM_data['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(EM_data)} {option}")
        EM_years = seispy.datetime_to_decimal_year(pre_EM['DATETIME'])
        ax4.scatter(EM_years, pre_EM['MAGNITUDE'], marker='s',
                    alpha=size_dict['point_alpha'], s=params['size'](pre_EM['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(EM_data)} {option}")
        i+=1

    ax3.set_ylabel('Magnitude')
    for ax in [ax3, ax4]:
        ax.set_xlabel('Year')
        # ax.legend()

    i=0
    for ax in [ax1, ax2, ax3, ax4]:
        ax.set_title(labels[i], loc='left')
        i+=1

    ax3.xaxis.set_ticks(range(2008, 2019,4))
    ax3.set_xticklabels(range(2008, 2019,4))
    ax4.xaxis.set_ticks(range(2010, 2012,1))
    ax4.set_xticklabels(range(2010, 2012,1))
    # ax1.legend(loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(loc='upper right', ncol=1, fontsize=12)

    box = [-116.5, -114.5, 31.9, 33.1] #[-116,-115, 32, 33]
    add_box_3857(box, ax=ax1, zorder=100, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['pink'])
    add_box_3857(box, ax=ax2, zorder=100, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['pink'])
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], ax=ax1, fontsize=size_dict['textsize'])
    lat_lon_tick_labels([-116, -115], [32, 33], ax=ax2, fontsize=size_dict['textsize'])
    
    ax1.axis(convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5]))
    ax2.axis(convert_extent_to_epsg3857([-116.6, -114.4, 31.8, 33.2]))

    # add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax1, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'], alpha=0.9)
    # plot_SCSN_reporting_region(ax=ax1, color=plot_color_dict['green'], zorder=10, linewidth=size_dict['linewidth'], alpha=0.9)

    for ax in [ax1, ax2]:
        cx.add_basemap(ax, source=source)
    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax1, zorder=1)
    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax2, zorder=1000)
    
    EM_legend = [Line2D([], [], color=selection_dict['Both']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[0], linestyle='None'),
                Line2D([], [], color=selection_dict['FET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[1], linestyle='None'),
                Line2D([], [], color=selection_dict['MDET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[2], linestyle='None'),
                Line2D([], [], color=selection_dict['Neither']['color'], marker='o', markersize=size_dict['legend_markersize']/2, label=EM_legend_labels[3], linestyle='None')
                    ]
    
    QTM_legend = [Line2D([], [], color=selection_dict['Both']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[0], linestyle='None'),
                Line2D([], [], color=selection_dict['FET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[1], linestyle='None'),
                Line2D([], [], color=selection_dict['MDET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[2], linestyle='None'),
                Line2D([], [], color=selection_dict['Neither']['color'], marker='o', markersize=size_dict['legend_markersize']/2, label=QTM_legend_labels[3], linestyle='None')
                ]

    # ax1.legend(handles=QTM_legend, loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(handles=EM_legend, loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=11)

    DDET_legend = [Line2D([], [], label='A: ID10832573', linestyle='None'),
                    Line2D([], [], label='B: ID14898996', linestyle='None')]
    # ax2.legend(handles=DDET_legend, loc='upper right', fontsize=20).set_zorder(2000)
    ax2.annotate('A: ID10832573', (-12905000, 3874000), fontsize=30, zorder=2000)
    ax2.annotate('B: ID14898996', (-12856000, 3896000), fontsize=30, zorder=2000)

    ax3.legend(handles=QTM_legend, loc='lower center', bbox_to_anchor=(0.78, 0.59), ncol=1)
    ax4.legend(handles=EM_legend, loc='lower center', bbox_to_anchor=(0.79, 0.59), ncol=1)
    ax4.set_yticklabels([])
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.15, hspace=0.25)
    plt.savefig(f"../outputs/figures/El_Mayor_Cucupah_selections.png")
    plt.show()

In [ ]:
El_Mayor_data.loc[El_Mayor_data['Selection']=='FET']

In [ ]:
DDET_mainshocks.loc[DDET_mainshocks['ID']==37301704].columns

In [ ]:
DDET_mainshocks.loc[DDET_mainshocks['ID']==37301704][['n_local_cat', 'n_local_cat_1yr', 'n_for_Mc_50', 'n_for_Mc_50_QTM_12', 'radii_50', 'Mc']]

## Investigating the El Mayor-Cucupah MDET threshold of 356 km - is it too large?
Does it exclude some candidate mainshocks that should be selected?

In [ ]:
El_Mayor = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['MAGNITUDE']>7].iloc[0]

EMC_MDET_threshold = 356.167877

El_Mayor_data = mainshocks_in_all_SCSN.loc[(mainshocks_in_all_SCSN['DATETIME'] > pd.Timestamp(2010,1,1)) &\
                                      (mainshocks_in_all_SCSN['DATETIME'] < pd.Timestamp(2011,4,4))].copy()

El_Mayor_data['km_to_EMC'] = seispy.calculate_distance_pyproj_vectorized(El_Mayor_data['LON'], El_Mayor_data['LAT'], El_Mayor_data['LON'],  El_Mayor_data['LAT'])
El_Mayor_data_restricted = El_Mayor_data.loc[El_Mayor_data['km_to_EMC']<EMC_MDET_threshold].copy()

size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

scale = 2
size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)
neither_relation = (lambda x: (5*scale + np.exp(1.1*x))/scale)
selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['yellow'], 'size':neither_relation, 'linewidth':0.25}
                    }

labels = [letter + ')' for letter in alphabet[:4]]

box_alpha = 0.2
with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(3,3)

    # ax1 = fig.add_subplot(gs[0, 0])
    # ax2 = fig.add_subplot(gs[0, 1])
    # ax3 = fig.add_subplot(gs[1, 0])
    # ax4 = fig.add_subplot(gs[1, 1])

    ax2 = fig.add_subplot(gs[:2, :])
    ax4 = fig.add_subplot(gs[2, :])

    i = 0
    QTM_legend_labels = []
    EM_legend_labels = []
    for option, params in selection_dict.items():
        EM_data = El_Mayor_data.loc[El_Mayor_data['Selection']==option].sort_values(by='MAGNITUDE', ascending=False)
        EM_data_restricted = El_Mayor_data_restricted.loc[El_Mayor_data_restricted['Selection']==option].sort_values(by='MAGNITUDE', ascending=False)
        mainshock_option_data = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']==option]

        if option in ['FET', 'MDET']:
            option = f"{option} only"
        QTM_legend_labels.append(f"{len(mainshock_option_data)} {option}")
        EM_legend_labels.append(f"{len(EM_data)} {option}")

        ax1.scatter(mainshock_option_data['geometry'].x, mainshock_option_data['geometry'].y,
            alpha=size_dict['point_alpha'], s=params['size'](mainshock_option_data['MAGNITUDE']), color=params['color'],
            zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
            label=f"{len(mainshock_option_data)} {option}")

        ax2.scatter(EM_data['geometry'].x, EM_data['geometry'].y, 
                    alpha=size_dict['point_alpha'], s=params['size'](EM_data['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(EM_data)} {option}")
        
        # radius_km = 10
        # new_LON, new_LAT = seispy.add_distance_to_position_pyproj(El_Mayor['LON'], El_Mayor['LAT'], radius_km, 0)
        # radius_degrees = new_LON - El_Mayor['LON']
        # circle_10km = Circle(EPSG_transformer(El_Mayor['LON'], El_Mayor['LAT']), radius=radius_degrees, facecolor='red', edgecolor='red', alpha=0.9, zorder=100)
        # ax2.add_patch(circle_10km)
        radius_km = 356.167877*1000	
        MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
        ax2.add_patch(MDET_circle)
        MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
        ax1.add_patch(MDET_circle)

        QTM_years = seispy.datetime_to_decimal_year(mainshock_option_data['DATETIME'])
        ax3.scatter(QTM_years, mainshock_option_data['MAGNITUDE'],
                    alpha=size_dict['point_alpha'], s=params['size'](mainshock_option_data['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(mainshock_option_data)} {option}")
        EM_years = seispy.datetime_to_decimal_year(EM_data_restricted['DATETIME'])
        ax4.scatter(EM_years, EM_data_restricted['MAGNITUDE'],
                    alpha=size_dict['point_alpha'], s=params['size'](EM_data_restricted['MAGNITUDE']), color=params['color'],
                    zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                    label=f"{len(EM_data)} {option}")
        i+=1

    ax3.set_ylabel('Magnitude')
    for ax in [ax3, ax4]:
        ax.set_xlabel('Year')
        # ax.legend()

    i=0
    for ax in [ax1, ax2, ax3, ax4]:
        ax.set_title(labels[i], loc='left')
        i+=1

    ax3.xaxis.set_ticks(range(2008, 2019,4))
    ax3.set_xticklabels(range(2008, 2019,4))
    ax4.xaxis.set_ticks(range(2010, 2012,1))
    ax4.set_xticklabels(range(2010, 2012,1))
    # ax1.legend(loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(loc='upper right', ncol=1, fontsize=12)

    box = [-116.5, -114.5, 31.9, 33.1] #[-116,-115, 32, 33]
    add_box_3857(box, ax=ax1, zorder=100, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['pink'])
    add_box_3857(box, ax=ax2, zorder=100, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['pink'])
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], ax=ax1, fontsize=size_dict['textsize'])
    lat_lon_tick_labels([-116, -115], [32, 33], ax=ax2, fontsize=size_dict['textsize'])
    
    ax1.axis(convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5]))
    # ax2.axis(convert_extent_to_epsg3857([-116.6, -114.4, 31.8, 33.2]))

    # add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax1, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'], alpha=0.9)
    # plot_SCSN_reporting_region(ax=ax1, color=plot_color_dict['green'], zorder=10, linewidth=size_dict['linewidth'], alpha=0.9)

    for ax in [ax1, ax2]:
        cx.add_basemap(ax, source=source)
    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax1, zorder=1)
    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax2, zorder=1000)
    
    EM_legend = [Line2D([], [], color=selection_dict['Both']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[0], linestyle='None'),
                Line2D([], [], color=selection_dict['FET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[1], linestyle='None'),
                Line2D([], [], color=selection_dict['MDET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=EM_legend_labels[2], linestyle='None'),
                Line2D([], [], color=selection_dict['Neither']['color'], marker='o', markersize=size_dict['legend_markersize']/2, label=EM_legend_labels[3], linestyle='None')
                    ]
    
    QTM_legend = [Line2D([], [], color=selection_dict['Both']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[0], linestyle='None'),
                Line2D([], [], color=selection_dict['FET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[1], linestyle='None'),
                Line2D([], [], color=selection_dict['MDET']['color'], marker='o', markersize=size_dict['legend_markersize'], label=QTM_legend_labels[2], linestyle='None'),
                Line2D([], [], color=selection_dict['Neither']['color'], marker='o', markersize=size_dict['legend_markersize']/2, label=QTM_legend_labels[3], linestyle='None')
                ]

    # ax1.legend(handles=QTM_legend, loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=12)
    # ax2.legend(handles=EM_legend, loc='upper center', bbox_to_anchor=(0.55, 1.3), ncol=2, fontsize=11)

    ax3.legend(handles=QTM_legend, loc='lower center', bbox_to_anchor=(0.78, 0.59), ncol=1)
    ax4.legend(handles=EM_legend, loc='lower center', bbox_to_anchor=(0.79, 0.59), ncol=1)
    ax4.set_yticklabels([])
    ax2.set_yticklabels([])
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.15, hspace=0.25)
    # plt.savefig(f"../outputs/figures/El_Mayor_Cucupah_MDET_threshold.png")
    plt.show()

## Plotting how close in space and time events are to the events which exclude them.
Is there a clear cutoff point?

Note: 
Event 10148002, M5.2, is not present in the QTM, but is a reason that a mainshock is excluded (not the only reason).  
I am therefore choosing to use all SCSN candidate events in making this plot, including event 10148002.

In [ ]:
FET = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']=='FET'].copy()
MDET = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']=='MDET'].copy()
Neither = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']=='Neither'].copy()
DDET = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']=='Both'].copy()
len(FET), len(MDET)
DDET

In [ ]:
T_days = 365
R_km = 250
tot = 0
for m in DDET.itertuples():
    mag_fours = mainshocks_in_all_SCSN.copy()
    mag_fours['km_to_mainshock'] = list(seispy.calculate_distance_pyproj_vectorized(m.LON, m.LAT, mainshocks_in_all_SCSN['LON'], mainshocks_in_all_SCSN['LAT']))
    mag_fours['time_to_mainshock'] = list((mainshocks_in_all_SCSN['DATETIME'] - m.DATETIME).apply(lambda x: x.total_seconds()/(3600*24)))
    nearby = mag_fours.loc[(mag_fours['time_to_mainshock']>-T_days) & (mag_fours['time_to_mainshock']<0) &\
                           (mag_fours['km_to_mainshock']<R_km) & (mag_fours['MAGNITUDE']>m.MAGNITUDE) ].copy()
    tot+=len(nearby)
    # print(m.ID, len(nearby))
print(tot)

In [ ]:
scale = 0.25
size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)
selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5, 'marker':'o', 'label':'DDET selects'},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5, 'marker':'v', 'label':'FET selects'},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5, 'marker':'^', 'label':'MDET selects'},
                    'Neither':{'zorder':10, 'color':plot_color_dict['yellow'], 'size':neither_relation, 'linewidth':0.25, 'marker':'<', 'label':'None select'}
                    }

fig = plt.figure()
ax = fig.add_subplot(111)

# exclusion_dict = {'FET':{'ID':[], 'cause':[], 'km':[], 'T':[], 'M':[]}}
selection_list = []
ID = []
cause = []
km = []
T = []
M = []
for selection, params in selection_dict.items():
    mainshocks = mainshocks_in_all_SCSN.loc[mainshocks_in_all_SCSN['Selection']==selection].copy()
    # mainshocks = mainshocks.iloc[:2].copy()
    print(selection, len(mainshocks))
    for m in mainshocks.itertuples():
        if selection == 'MDET':
            exclusion_causes_IDs = ast.literal_eval(m.Moutote_excluded_by)
        elif selection =='FET':
            exclusion_causes_IDs = ast.literal_eval(m.TR_excluded_by)
        elif selection =='Neither':
            MDET_exclusion_causes_IDs = ast.literal_eval(m.TR_excluded_by)
            FET_exclusion_causes_IDs = ast.literal_eval(m.Moutote_excluded_by)
            exclusion_causes_IDs = MDET_exclusion_causes_IDs + FET_exclusion_causes_IDs
            exclusion_causes_IDs = set(exclusion_causes_IDs)
            both_exclude = [ID for ID in exclusion_causes_IDs if (ID in MDET_exclusion_causes_IDs) & (ID in FET_exclusion_causes_IDs)]
            FET_only_excludes = [ID for ID in exclusion_causes_IDs if (ID not in MDET_exclusion_causes_IDs) & (ID in FET_exclusion_causes_IDs)]
            MDET_only_excludes = [ID for ID in exclusion_causes_IDs if (ID in MDET_exclusion_causes_IDs) & (ID not in FET_exclusion_causes_IDs)]
            # print(f"both_exclude: {len(both_exclude)}")
            # print(f"FET_only_excludes: {len(FET_only_excludes)}")
            # print(f"MDET_only_excludes: {len(MDET_only_excludes)}")
        elif selection=='Both':
                mag_fours = mainshocks_in_all_SCSN.copy()
                mag_fours['km_to_mainshock'] = list(seispy.calculate_distance_pyproj_vectorized(m.LON, m.LAT, mainshocks_in_all_SCSN['LON'], mainshocks_in_all_SCSN['LAT']))
                mag_fours['time_to_mainshock'] = list((mainshocks_in_all_SCSN['DATETIME'] - m.DATETIME).apply(lambda x: x.total_seconds()/(3600*24)))
                nearby = mag_fours.loc[(mag_fours['time_to_mainshock']>-T_days) & (mag_fours['time_to_mainshock']<0) &\
                                       (mag_fours['km_to_mainshock']<R_km) & (mag_fours['MAGNITUDE']>m.MAGNITUDE) ].copy()           
                exclusion_causes_IDs = list(nearby['ID'])

        exclusion_causes = SCSN_mainshocks.loc[SCSN_mainshocks['ID'].isin(exclusion_causes_IDs)].copy()
        # print(len(exclusion_causes), print(len(exclusion_causes['ID'].unique())))
        # if all(ID in exclusion_causes_IDs for ID in mainshocks_in_all_SCSN['ID'])==False:
        #     print('False')
        #     KeyboardInterrupt
        km_to_mainshock = list(seispy.calculate_distance_pyproj_vectorized(m.LON, m.LAT, exclusion_causes['LON'], exclusion_causes['LAT']))
        time_to_mainshock = list((exclusion_causes['DATETIME'] - m.DATETIME).apply(lambda x: x.total_seconds()/(3600*24)))
        # print(len(exclusion_causes), len(km_to_mainshock))
        # if len(exclusion_causes)!= len(km_to_mainshock):
        #     print('Double False')
        #     print(exclusion_causes)
        #     KeyboardInterrupt
        selection_list.extend([selection]*len(exclusion_causes_IDs))
        ID.extend([m.ID]*len(exclusion_causes_IDs))
        cause.extend(exclusion_causes_IDs)
        km.extend(km_to_mainshock)
        T.extend(time_to_mainshock)
        M.extend(list(exclusion_causes['MAGNITUDE']))
    # FET = pd.DataFrame.from_dict(exclusion_dict['FET'])
    # FET
        # ax.scatter(time_to_mainshock, km_to_mainshock, color=plot_color_dict['orange'])
        # first_cause = exclusion_causes.iloc[0]
        # distance_to_first_cause = 
        # print(len(exclusion_causes))
    # plt.show()
exclusion_dict = {'selection':selection_list, 'ID':ID, 'cause':cause, 'km':km, 'T':T, 'M':M}
exclusion_df = pd.DataFrame(exclusion_dict)

for selection, params in selection_dict.items():
    df = exclusion_df.loc[exclusion_df['selection']==selection].copy()
    print(selection, len(df))
    ax.scatter(df['T'], df['km'], color=params['color'], s=[size_relation(M) for M in df['M']], alpha=0.8, ec='white', zorder=params['zorder'], label=params['label'],
               marker=params['marker'])
ax.set_xlabel('Time (days)')
ax.set_ylabel('Distance (km)')
ax.legend()
ax.axhline(10, label='FET 10 km boundary', color=plot_color_dict['orange'], linewidth=2)
plt.tight_layout()
plt.savefig('../outputs/figures/mainshock_selection_mess.png')
plt.show()

### The event in the SCSN but not the QTM


In [ ]:
difference = [x for x in all_exclusion_IDs if x not in list(mainshocks_in_all_SCSN['ID'])]
difference
seispy.find_event_in_catalog(10148002, SCSN_mainshocks)

# x. Does selection state for mainshocks change between catalogs? (unused?)

In [ ]:
merged_mainshocks = pd.merge(pd.merge(SCSN_mainshocks, QTM_12_mainshocks, on='ID', suffixes=('', '_QTM_12')), QTM_9_5_mainshocks, on='ID', suffixes=('_SCSN', '_QTM_9_5'))
differing_selections = merged_mainshocks.loc[merged_mainshocks['Selection_QTM_12']!=merged_mainshocks['Selection_SCSN']].copy()
differing_selections[['ID', 'Selection_SCSN', 'Selection_QTM_12', 'DATETIME_QTM_12', 'TR_excluded_by_SCSN', 'TR_excluded_by_QTM_12']].sort_values(by=['Selection_SCSN', 'Selection_QTM_12'])

In [ ]:
bbox_args = dict(boxstyle="round,pad=0.5", fc='white', alpha=1)
labels = [letter + ')' for letter in alphabet[:4]]

scale = 2
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':1*scale,
             'legend_markersize':10,
             'neither_markersize':5
             }

# size_relation = (lambda x: 6*x**2)
size_relation = (lambda x: np.exp(1.1*x))
neither_relation = (lambda x: 10)

cat_options_dict = {'SCSN':{'zorder':13, 'color':'teal', 'size':size_relation, 'linewidth':0.5, 'label':'SCSN'},
                    'QTM_12':{'zorder':12, 'color':'orange', 'size':size_relation, 'linewidth':0.5, 'label':'QTM 12'},
                    'QTM_9_5':{'zorder':11, 'color':'purple', 'size':size_relation, 'linewidth':0.5, 'label':'QTM 9.5'},
                    }

plot_options_dict = {'Both':{'zorder':11, 'color':'teal', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'FET':{'zorder':12, 'color':'orange', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'MDET':{'zorder':13, 'color':'purple', 'size':size_relation, 'linewidth':0.5, 'ec':'white'},
                    'Neither':{'zorder':10, 'color':'grey', 'size':size_relation, 'linewidth':0.5, 'ec':'red'}
                    }

bbox_args = dict(boxstyle="round,pad=0.5", fc='white', alpha=1)

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'],
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']*scale/6}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(2,2)
    ax = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[1,0])
    ax4 = fig.add_subplot(gs[1,1])

    legend_labels = []
    difference_region = [-116.5, -114.5, 31.5, 33]
    cat = 'SCSN'
    text_x, text_y = -114.55, 32.85
    ax.annotate(cat, EPSG_transformer((text_x, text_y))[0], zorder=5, bbox=bbox_args, ha='right', fontsize=size_dict['textsize'])
    main_cat = mainshock_dict[cat].copy()
    main_cat = seispy.restrict_catalogue_geographically(main_cat, region=difference_region)
    main_cat = main_cat.loc[(main_cat['DATETIME'] > dt.datetime(2008,1,1)) & (main_cat['DATETIME'] < dt.datetime(2012,6,1))]
    plot_params = cat_options_dict[cat]

    ax.scatter(main_cat["geometry"].x, main_cat["geometry"].y, 
                color='grey', zorder=10, alpha=0.5,
                s=plot_params['size'](main_cat['MAGNITUDE'])*scale, ec='white', linewidth=0.25)
    years = seispy.datetime_to_decimal_year(main_cat['DATETIME'])
    ax2.scatter(years, main_cat['MAGNITUDE'], s=plot_params['size'](main_cat['MAGNITUDE'])*scale, alpha=0.5,
                color='grey', ec='white', linewidth=0.25)
    ax2.set_xlim(2008, 2012.5)

    legend_labels = []
    c = 0
    for selection, item in plot_options_dict.items():
        mainshock_selection = differing_selections.loc[differing_selections[f'Selection_{cat}']==selection].copy()
        ax.scatter(mainshock_selection[f'geometry_{cat}'].x, mainshock_selection[f'geometry_{cat}'].y, 
                   alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}'])*scale,
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        years = seispy.datetime_to_decimal_year(mainshock_selection[f"DATETIME_{cat}"])
        ax2.scatter(years, mainshock_selection[f'MAGNITUDE_{cat}'],
                   alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}'])*scale,
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        if selection in ['FET', 'MDET']:
            selection = f"{selection} only"
        legend_labels.append(f"{selection}: {len(mainshock_selection)}")
        c+=1

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[1]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[2]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_color_dict['grey'], markeredgecolor=item['ec'], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[3], linestyle='None', linewidth=0.5)
                    ]

    ax.legend(handles=legend_handles, loc='lower left', framealpha=1, bbox_to_anchor=(-0.01, 0.01), ncol=2, fontsize=size_dict['textsize']/2,
            handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)

    cat = 'QTM_12'
    ax3.annotate('QTM 12', EPSG_transformer((text_x, text_y))[0], zorder=5, bbox=bbox_args, ha='right', fontsize=size_dict['textsize'])
    main_cat = mainshock_dict[cat].copy()
    main_cat = seispy.restrict_catalogue_geographically(main_cat, region=difference_region)
    main_cat = main_cat.loc[(main_cat['DATETIME'] > dt.datetime(2008,1,1)) & (main_cat['DATETIME'] < dt.datetime(2012,6,1))]
    plot_params = cat_options_dict[cat]
    ax3.scatter(main_cat["geometry"].x, main_cat["geometry"].y, 
                color='grey', zorder=10, alpha=0.5,
                s=plot_params['size'](main_cat['MAGNITUDE'])*scale, ec='white', linewidth=0.25)
    years = seispy.datetime_to_decimal_year(main_cat['DATETIME'])
    ax4.scatter(years, main_cat['MAGNITUDE'], s=plot_params['size'](main_cat['MAGNITUDE'])*scale, alpha=0.5,
                color='grey', ec='white', linewidth=0.25)
    ax4.set_xlim(2008, 2012.5)
    ax4.set_xticks([2008, 2010, 2012])
    ax4.set_xticklabels([2008, 2010, 2012])
    legend_labels = []
    c = 0
    for selection, item in plot_options_dict.items():
        mainshock_selection = differing_selections.loc[differing_selections[f'Selection_{cat}']==selection].copy()
        ax3.scatter(mainshock_selection[f'geometry_{cat}'].x, mainshock_selection[f'geometry_{cat}'].y, 
                   alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}'])*scale,
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        years = seispy.datetime_to_decimal_year(mainshock_selection[f"DATETIME_{cat}"])
        ax4.scatter(years, mainshock_selection[f'MAGNITUDE_{cat}'],
                   alpha=0.95, s=item['size'](mainshock_selection[f'MAGNITUDE_{cat}'])*scale,
                    color=plot_color_dict[item['color']],
                    zorder=item['zorder'], ec=item['ec'], linewidth=item['linewidth'])
        if selection in ['FET', 'MDET']:
            selection = f"{selection} only"
        legend_labels.append(f"{selection}: {len(mainshock_selection)}")
        c+=1

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[1]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=f"{legend_labels[2]}", linestyle='None', linewidth=0.5),
                    Line2D([], [], color=plot_color_dict['grey'], markeredgecolor=item['ec'], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[3], linestyle='None', linewidth=0.5)
                    ]

    ax3.legend(handles=legend_handles, loc='lower left', framealpha=1, bbox_to_anchor=(-0.01, 0.01), ncol=2, fontsize=size_dict['textsize']/2,
            handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5).set_zorder(100)
    i=0
    for axs in [ax, ax2, ax3, ax4]:        
        axs.set_title(labels[i], loc='left')
        i+=1

    for axs in [ax2,ax4]:
        axs.set_ylabel('Magnitude')
        axs.set_xlabel('Year')

    source = cx.providers.Esri.WorldTerrain
    for axs in [ax, ax3]:
    # add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15)
        lat_lon_tick_labels([-116, -115, -114], [32, 33], ax=axs, fontsize=size_dict['textsize'])
    # plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['teal'], zorder=16)
        extent = convert_extent_to_epsg3857([-116.5, -114.5, 31.5, 33])
        axs.axis(extent)
        cx.add_basemap(axs, source=source)
        fault_gdf.plot(color='black', linewidth=0.5, ax=axs, zorder=20)

    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    plt.savefig('../outputs/figures/differing_selections.png')
    plt.show()

# x Mainshock figure (unused)
Change so legend is at bottom of map like section 7.  

In [ ]:
# mainshock_dict = {}
# for name, data in catalogue_dict.items():
#     mainshock_file = pd.read_csv(f'../data/{name}/mainshocks.csv')
#     string_to_datetime_df(mainshock_file)
#     # mainshock_file.to_csv(f'../data/{name}/mainshocks.csv', index=False)
#     mainshock_file = df_to_gdf(mainshock_file)
#     mainshock_dict[name] = mainshock_file

# mainshocks_in_all = pd.read_csv('../data/QTM_12/mainshocks_in_all.csv')
mainshocks_in_all = mainshocks_in_all_dict['QTM_12']
# mainshocks_in_all['DATETIME'] = pd.to_datetime(mainshocks_in_all['DATETIME'], format = '%Y-%m-%d %H:%M:%S.%f')

bbox_args = dict(boxstyle="round,pad=0.05", fc='white', alpha=0.8)

scale = 2 #3
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

size_relation = (lambda x: 5*scale + np.exp(1.1*x))
cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7, 'ref':'Hutton et al., 2010', 'label': 'SCSN'},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 12'},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 9.5'},
                    'Combined':{'label': 'Merged'}
                    }

neither_relation = (lambda x: scale_eq_marker(x)/2)

selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'Neither':{'zorder':15, 'color':plot_color_dict['grey'], 'size':neither_relation, 'linewidth':0.25}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):

  fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)

  i = 0
  labels = [letter + ')' for letter in alphabet[:4]]
  mainshock_dict.update({'Combined':mainshocks_in_all})
  # [mainshock_dict.update({key:df_to_gdf(df)}) for key, df in mainshock_dict.items()]
  for name, mainshocks in mainshock_dict.items():
    cat_params = cat_options_dict[name]
    ax = fig.add_subplot(2,2,i+1)
    ax.set_title(labels[i], loc='left')
    ax.annotate(cat_params['label'], EPSG_transformer(size_dict['label_pos'])[0], fontsize=size_dict['textsize'], zorder=50, bbox=bbox_args, ha='left')

    legend_labels = []
    for selection, params in selection_dict.items():
      mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()      
      ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=size_dict['point_alpha'],
                  s=params['size'](mainshock_selection['MAGNITUDE']), color=params['color'],
                  zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                  label=f"{selection}: {len(mainshock_selection)}")
      # legend_labels.append(f"{selection}: {len(mainshock_selection)}")
      if selection in ['FET', 'MDET']:
        selection = f"{selection} only"
      legend_labels.append(f"{len(mainshock_selection)} {selection}")

    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax, zorder=1)
    source = cx.providers.Esri.WorldTerrain
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=15, linewidth=size_dict['linewidth'], alpha=0.9)
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], fontsize=size_dict['textsize'], ax=ax)
    plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['green'], zorder=10, linewidth=size_dict['linewidth'], alpha=0.9)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)
    cx.add_basemap(ax, source=source)

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None'),
                      Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[1], linestyle='None'),
                      Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[2], linestyle='None'),
                      Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['legend_markersize']/1.5, label=legend_labels[3], linestyle='None')
                      ]
    
    # ax.legend(handles=legend_handles, loc='lower left', bbox_to_anchor=(0, 0), ncol=2, framealpha=0.7,
    #           handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    ax.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.585, 1.24), ncol=2, framealpha=0.7,
              handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    if i==0:
      ax.set_xticklabels([])
    elif i==1:
      ax.set_xticklabels([])
      ax.set_yticklabels([])
    elif i==3:
      ax.set_yticklabels([])

    i+=1

  plt.tight_layout()
  fig.subplots_adjust(wspace=-0.3, hspace=0.25)
  # plt.savefig(f"../outputs/figures/mainshock_selection_map_new.png", bbox_inches='tight')
  plt.show()

# 6. Investigating mainshock locations and the SCSN reporting region.
I should really drop the QTM 12 and 9.5 panels, just have SCSN and QTM only. I could also show time series? Idk, this feels like a supplementary plot.

In [ ]:
mainshock_dict = {}
for name, data in catalogue_dict.items():
    mainshock_file = pd.read_csv(f'../data/{name}/mainshocks.csv')
    string_to_datetime_df(mainshock_file)
    # mainshock_file.to_csv(f'../data/{name}/mainshocks.csv', index=False)
    mainshock_file = df_to_gdf(mainshock_file)
    mainshock_dict[name] = mainshock_file

SCSN_mainshocks = mainshock_dict['SCSN'].copy()
QTM_9_5_mainshocks = mainshock_dict['QTM_9_5'].copy()
QTM_12_mainshocks = mainshock_dict['QTM_12'].copy()

QTM_9_5_only_mainshocks = QTM_9_5_mainshocks.loc[~(QTM_9_5_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
                                                 ~(QTM_9_5_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

QTM_12_only_mainshocks = QTM_12_mainshocks.loc[~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID'])) &\
                                               ~(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID']))].copy  ()

SCSN_only_mainshocks = SCSN_mainshocks.loc[~(SCSN_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                           ~(SCSN_mainshocks['ID'].isin(QTM_12_mainshocks['ID']))].copy()

QTM_only_mainshocks =  QTM_12_mainshocks.loc[(QTM_12_mainshocks['ID'].isin(QTM_9_5_mainshocks['ID'])) &\
                                             ~(QTM_12_mainshocks['ID'].isin(SCSN_mainshocks['ID']))].copy()

mainshocks_in_one_dict = {'QTM_9_5':QTM_9_5_only_mainshocks,
                          'QTM_12':QTM_12_only_mainshocks,
                          'SCSN':SCSN_only_mainshocks,
                          'QTM':QTM_only_mainshocks}

for key, item in mainshock_dict.items():
    mainshock_dict[key] = df_to_gdf(item)


for key, item in mainshocks_in_one_dict.items():
    item['catalogue'] = key
    mainshock_dict[key] = df_to_gdf(item)

for key, item in mainshocks_in_one_dict.items():
    print(len(item))

In [ ]:
mainshocks_in_one_dict.keys()

In [ ]:
scale = 2 #3
size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65)
             }

size_relation = (lambda x: 5*scale + np.exp(1.1*x))
cat_options_dict = {'SCSN':{'zorder':13, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7, 'ref':'Hutton et al., 2010', 'label': 'SCSN'},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 12'},
                    'QTM_9_5':{'zorder':11, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 9.5'},
                    'Combined':{'label': 'Merged'},
                    'QTM':{'zorder':11, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 'ref':'Ross et al., 2019', 'label': 'QTM 9.5'},
                    }

selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'Neither':{'zorder':15, 'color':plot_color_dict['grey'], 'size':neither_relation, 'linewidth':0.25}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):

  fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)

  i = 0
  labels = [letter + ')' for letter in alphabet[:4]]

  for name, mainshocks in mainshocks_in_one_dict.items():
    cat_params = cat_options_dict[name]
    ax = fig.add_subplot(2,2,i+1)
    ax.set_title(labels[i], loc='left')
    ax.annotate(cat_params['label'], EPSG_transformer(size_dict['label_pos'])[0], fontsize=size_dict['textsize'], zorder=50, bbox=bbox_args, ha='left')

    legend_labels = []
    for selection, params in selection_dict.items():
      mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()      
      ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=size_dict['point_alpha'],
                  s=params['size'](mainshock_selection['MAGNITUDE']), color=params['color'],
                  zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                  label=f"{selection}: {len(mainshock_selection)}")
      # legend_labels.append(f"{selection}: {len(mainshock_selection)}")
      if selection in ['FET', 'MDET']:
        selection = f"{selection} only"
      legend_labels.append(f"{len(mainshock_selection)} {selection}")

    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax, zorder=1)
    source = cx.providers.Esri.WorldTerrain
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=1, linewidth=size_dict['linewidth'], alpha=0.9)
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], fontsize=size_dict['textsize'], ax=ax)
    plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['green'], zorder=1, linewidth=size_dict['linewidth'], alpha=0.9)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)
    cx.add_basemap(ax, source=source)

    legend_handles = [Line2D([], [], color=plot_colors[0], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[0], linestyle='None'),
                      Line2D([], [], color=plot_colors[1], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[1], linestyle='None'),
                      Line2D([], [], color=plot_colors[2], marker='o', markersize=size_dict['legend_markersize'], label=legend_labels[2], linestyle='None'),
                      Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['legend_markersize']/1.5, label=legend_labels[3], linestyle='None')
                      ]
    
    # ax.legend(handles=legend_handles, loc='lower left', bbox_to_anchor=(0, 0), ncol=2, framealpha=0.7,
    #           handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    ax.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.585, 1.24), ncol=2, framealpha=0.7,
              handletextpad=0.5, borderpad=0.5, labelspacing=0.5, handlelength=0.5, columnspacing=0.75).set_zorder(1000)
    
    if i==0:
      ax.set_xticklabels([])
    elif i==1:
      ax.set_xticklabels([])
      ax.set_yticklabels([])
    elif i==3:
      ax.set_yticklabels([])

    i+=1

  plt.tight_layout()
  fig.subplots_adjust(wspace=-0.3, hspace=0.25)
  # plt.savefig(f"../outputs/figures/one_cat_only_mainshocks.png", bbox_inches='tight')
  plt.show()

In [ ]:
all_mainshocks_in_one  = pd.concat([item for key, item in mainshocks_in_one_dict.items()])
all_mainshocks_in_one

In [ ]:
m = mainshocks_in_one_dict['SCSN']
m
# m.loc[m['Selection']!='Neither']['ID'].values

In [ ]:
cat_options_dict = {'SCSN':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'Mc':1.7,
                             'ref':'Hutton et al., 2010', 'label': 'SCSN', 'ec':'white', 'marker':'o'},
                    'QTM_12':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'Mc':0.3, 
                              'ref':'Ross et al., 2019', 'label': 'QTM 12', 'ec':'white', 'marker':'D'},
                    'QTM_9_5':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'Mc':0.3, 
                               'ref':'Ross et al., 2019', 'label': 'QTM 9.5', 'ec':plot_color_dict['pink'], 'marker':'D'},
                    'Combined':{'label': 'Merged'},
                    'QTM':{'zorder':10, 'color':plot_color_dict['orange'], 'size':size_relation, 'ec':'white', 'marker':'D'}
                    }


selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':scale_eq_marker, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['grey'], 'size':neither_relation, 'linewidth':0.25}
                    }

with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):
    fig = plt.figure(figsize=(16,14), dpi=300)
    ax = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)

    for name, mainshocks in mainshocks_in_one_dict.items():
        cat_params = cat_options_dict[name]
        legend_labels = []
        for selection, params in selection_dict.items():
            mainshock_selection = mainshocks.loc[mainshocks['Selection']==selection].copy()      
            ax.scatter(mainshock_selection['geometry'].x, mainshock_selection['geometry'].y, alpha=size_dict['point_alpha'],
                        s=params['size'](mainshock_selection['MAGNITUDE'])*10, color=params['color'], marker=cat_params['marker'],
                        zorder=params['zorder'], ec=cat_params['ec'], linewidth=size_dict['edge_width']*2,
                        label=f"{selection}: {len(mainshock_selection)}")
            years = seispy.datetime_to_decimal_year(mainshock_selection['DATETIME'])
            ax2.scatter(years, mainshock_selection['MAGNITUDE'],
                        alpha=size_dict['point_alpha'], s=params['size'](mainshock_selection['MAGNITUDE'])*4, color=params['color'],
                        marker=cat_params['marker'],
                        zorder=cat_params['zorder'], ec=cat_params['ec'], linewidth=size_dict['edge_width']*2,
                        label=f"{len(EM_data)} {option}")

    fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax, zorder=1)
    source = cx.providers.Esri.WorldTerrain
    add_box_3857([-118.8, -115.4, 32.68, 36.2], ax=ax, color=plot_color_dict['pink'], zorder=1, linewidth=size_dict['linewidth'], alpha=0.9)
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], fontsize=size_dict['textsize'], ax=ax)
    plot_SCSN_reporting_region(ax=ax, color=plot_color_dict['green'], zorder=1, linewidth=size_dict['linewidth'], alpha=0.9)
    big_map_extent = convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5])
    ax.axis(big_map_extent)
    cx.add_basemap(ax, source=source)

    ax2.set_xticks(range(2008, 2019,4))
    ax2.set_xticklabels(range(2008, 2019,4))

    ax2.scatter(seispy.datetime_to_decimal_year(mainshocks_in_all['DATETIME']), mainshocks_in_all['MAGNITUDE'], color='grey', zorder=0, alpha=0.5)
    ax2.set_ylabel('Magnitude')
    ax2.set_xlabel('Year')

    plt.show()

# 10. Comparing my mainshock catalog to T&R and Moutote's

How many of Moutote's mainshocks does my FET method select? Do I find more?  
How many of T&R's mainshocks does my MDET method select? Do I find more?  
Why?

In [ ]:
TR_mainshock_IDs = np.array([
    14383980, 15200401, 37374687, 15481673, 14408052, 15296281, 10410337, 15520985, 15189073, 10370141, 
    14601172, 11413954, 10527789, 15476961, 37507576, 15475329, 37510616, 14418600, 14898996, 37526424, 
    15447161, 11339042, 11373458, 10530013, 14571828, 37301704, 11001205, 14600292, 37298672, 10321561, 
    15226257, 15507801, 11006189, 14396336, 10489253, 15223417, 37299263, 15014900, 14403732, 15071220, 
    37166079, 14406304, 37644544, 15153497, 15267105, 37243591
])

TR_mainshocks = SCSN.loc[SCSN['ID'].isin(TR_mainshock_IDs)].copy()

low_Mc_region = [-118.8, -115.4, 32.68, 36.2]

Moutote_mainshock_ETAS_params = pd.read_csv('../data/Moutote/Moutote_ETAS_params.txt')
Moutote_mainshock_ETAS_params.rename(columns={'#mainshockID':'ID'}, inplace=True)
# Moutote_mainshock_ETAS_params

for key, df in mainshock_dict.items():
    df['In_Moutote'] = df['ID'].isin(Moutote_mainshock_ETAS_params['ID'])
    df['In_Moutote'] = df['ID'].isin(Moutote_mainshock_ETAS_params['ID'])

    df['In_T&R'] = df['ID'].isin(TR_mainshock_IDs)
    df['In_T&R'] = df['ID'].isin(TR_mainshock_IDs)

SCSN_mainshocks = mainshock_dict['SCSN']
QTM_12_mainshocks = mainshock_dict['QTM_12']
QTM_9_5_mainshocks = mainshock_dict['QTM_9_5']

good_mainshocks['In_Moutote'] = good_mainshocks['ID'].isin(Moutote_mainshock_ETAS_params['ID'])
good_mainshocks['In_Moutote'] = good_mainshocks['ID'].isin(Moutote_mainshock_ETAS_params['ID'])

good_mainshocks['In_T&R'] = good_mainshocks['ID'].isin(TR_mainshock_IDs)
good_mainshocks['In_T&R'] = good_mainshocks['ID'].isin(TR_mainshock_IDs)
good_mainshocks

## My 74 from my merged catalog vs past catalogs

I include 46/53 of Moutote's catalog, and 36/46 of T&Rs.  

There are seven of Moutote's which I exclude, 6 because they are not selected by the MDET method, 1 because it is not selected by my FET method.  



In [ ]:
len(good_mainshocks.loc[good_mainshocks['In_Moutote']==True]), len(good_mainshocks.loc[good_mainshocks['In_T&R']==True]), len(good_mainshocks.loc[good_mainshocks['In_Moutote']==False]), len(good_mainshocks.loc[good_mainshocks['In_T&R']==False]), 

In [ ]:
Moutote_mainshock_ETAS_params

In [ ]:
in_Moutote_not_mine = Moutote_mainshock_ETAS_params.loc[~Moutote_mainshock_ETAS_params['ID'].isin(good_mainshocks['ID'])].copy()
in_Moutote_not_mine

In [ ]:
SCSN_mainshocks.loc[SCSN_mainshocks['ID'].isin(in_Moutote_not_mine['ID'])][['Selection']]

In [ ]:
len(TR_mainshocks)

In [ ]:
in_TR_not_mine = TR_mainshocks.loc[~TR_mainshocks['ID'].isin(good_mainshocks['ID'])]
in_TR_not_mine

In [ ]:
SCSN_mainshocks.loc[SCSN_mainshocks['ID'].isin(in_TR_not_mine['ID'])]['Selection']

## TR mainshocks

In [ ]:
my_MDET_SCSN = SCSN_mainshocks.loc[(SCSN_mainshocks['TR_method']=='Selected')]
my_MDET_SCSN

In [ ]:
my_MDET_low_Mc_SCSN = seispy.restrict_catalogue_geographically(my_MDET_SCSN, region=low_Mc_region)
my_MDET_low_Mc_SCSN

In [ ]:
my_extra_MDET_SCSN = my_MDET_low_Mc_SCSN.loc[my_MDET_low_Mc_SCSN['In_T&R']==False]
my_extra_MDET_SCSN

In [ ]:
set(my_MDET_low_Mc_SCSN['ID']) - set(TR_mainshock_IDs)

In [ ]:
set(TR_mainshock_IDs) - set(my_MDET_low_Mc_SCSN['ID'])

In [ ]:
SCSN_mainshocks.loc[SCSN_mainshocks['ID'].isin([10489253, 15153497])]

In [ ]:
missed_in_SCSN = QTM_9_5_mainshocks.loc[QTM_9_5_mainshocks['ID'].isin([10489253, 15153497])].copy()
missed_in_SCSN

In [ ]:
my_MDET_QTM_9_5 = QTM_9_5_mainshocks.loc[(QTM_9_5_mainshocks['TR_method']=='Selected')]
print(len(my_MDET_QTM_9_5))
my_MDET_low_Mc_QTM_9_5 = seispy.restrict_catalogue_geographically(my_MDET_QTM_9_5, region=low_Mc_region)
print(len(my_MDET_low_Mc_QTM_9_5))
my_extra_MDET_QTM_9_5 = my_MDET_low_Mc_QTM_9_5.loc[my_MDET_low_Mc_QTM_9_5['In_T&R']==False]
len(set(TR_mainshock_IDs) - set(my_MDET_low_Mc_QTM_9_5['ID']))
len(set(my_MDET_low_Mc_QTM_9_5['ID']) - set(TR_mainshock_IDs))

In [ ]:
my_MDET_QTM_12 = QTM_12_mainshocks.loc[(QTM_12_mainshocks['TR_method']=='Selected')]
print(len(my_MDET_QTM_12))
my_MDET_low_Mc_QTM_12 = seispy.restrict_catalogue_geographically(my_MDET_QTM_12, region=low_Mc_region)
print(len(my_MDET_low_Mc_QTM_12))
len(set(TR_mainshock_IDs) - set(my_MDET_low_Mc_QTM_12['ID']))
len(set(my_MDET_low_Mc_QTM_12['ID']) - set(TR_mainshock_IDs))

In [ ]:
scale = 2
size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)

size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65),
             'size':(lambda x: (5*scale + np.exp(1.1*x))*scale),
             'zorder':1
             }

params=size_dict

neither_relation = (lambda x: (5*scale + np.exp(1.1*x))/scale)
selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['yellow'], 'size':neither_relation, 'linewidth':0.25}
                    }

labels = [letter + ')' for letter in alphabet[:4]]

box_alpha = 0.2
with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(3,3)

    ax2 = fig.add_subplot(gs[:2, :])
    ax4 = fig.add_subplot(gs[2, :])

    ax2.scatter(my_MDET_QTM_9_5['geometry'].x, my_MDET_QTM_9_5['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](my_MDET_QTM_9_5['MAGNITUDE']), color=plot_color_dict['grey'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                label=f"All in QTM 9.5: {len(my_MDET_QTM_9_5)}")
    ax2.scatter(my_MDET_low_Mc_QTM_9_5['geometry'].x, my_MDET_low_Mc_QTM_9_5['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](my_MDET_low_Mc_QTM_9_5['MAGNITUDE']), color=plot_color_dict['teal'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                label=fr"In box: {len(my_MDET_low_Mc_QTM_9_5)}")
    ax2.scatter(missed_in_SCSN['geometry'].x, missed_in_SCSN['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](missed_in_SCSN['MAGNITUDE']), color=plot_color_dict['pink'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'],
                label=f"Outside box in SCSN: {len(missed_in_SCSN)}")
    ax2.scatter(my_extra_MDET_QTM_9_5['geometry'].x, my_extra_MDET_QTM_9_5['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](my_extra_MDET_QTM_9_5['MAGNITUDE']), color=plot_color_dict['orange'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'],
                label=f"Extra: {len(my_extra_MDET_QTM_9_5)}")
        
    # radius_km = 356.167877*1000	
    # MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
    # ax2.add_patch(MDET_circle)
    # MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
    # ax1.add_patch(MDET_circle)

    ax4.scatter(seispy.datetime_to_decimal_year(my_MDET_QTM_9_5['DATETIME']), my_MDET_QTM_9_5['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](my_MDET_QTM_9_5['MAGNITUDE']), color=plot_color_dict['grey'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(my_MDET_low_Mc_QTM_9_5['DATETIME']), my_MDET_low_Mc_QTM_9_5['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](my_MDET_low_Mc_QTM_9_5['MAGNITUDE']), color=plot_color_dict['teal'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(missed_in_SCSN['DATETIME']), missed_in_SCSN['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](missed_in_SCSN['MAGNITUDE']), color=plot_color_dict['pink'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(my_extra_MDET_QTM_9_5['DATETIME']), my_extra_MDET_QTM_9_5['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](my_extra_MDET_QTM_9_5['MAGNITUDE']), color=plot_color_dict['orange'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'])

    ax4.set_ylabel('Magnitude')
    ax4.set_xlabel('Year')
    ax2.set_title('a)', loc='left')
    ax4.set_title('b)', loc='left')
    ax4.xaxis.set_ticks(range(2008, 2019,4))
    ax4.set_xticklabels(range(2008, 2019,4))

    box = low_Mc_region
    add_box_3857(box, ax=ax2, zorder=1, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['brown'])
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], ax=ax2, fontsize=size_dict['textsize'])
    
    ax2.axis(convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5]))

    cx.add_basemap(ax2, source=source)
    # fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax2, zorder=1000)
    
    QTM_legend = [Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['legend_markersize'], label=f"All in QTM 9.5: {len(my_MDET_QTM_9_5)}", linestyle='None'),
                Line2D([], [], color=plot_color_dict['teal'], marker='o', markersize=size_dict['legend_markersize'], label=fr"In box: {len(my_MDET_low_Mc_QTM_9_5)}", linestyle='None'),
                Line2D([], [], color=plot_color_dict['pink'], marker='o', markersize=size_dict['legend_markersize'], label=f"Outside box in SCSN: {len(missed_in_SCSN)}", linestyle='None'),
                Line2D([], [], color=plot_color_dict['orange'], marker='o', markersize=size_dict['legend_markersize'], label=f"Extra: {len(my_extra_MDET_QTM_9_5)}", linestyle='None')
                ]

    ax2.legend(handles=QTM_legend, loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
    # ax2.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
    # pos = ax2.get_position()
    # new_pos = [pos.x0 - 0.7, pos.y0, pos.width, pos.height]
    # ax2.set_position(new_pos)
    # ax4.legend(handles=EM_legend, loc='lower center', bbox_to_anchor=(0.79, 0.59), ncol=1)
    plt.tight_layout()
    # fig.subplots_adjust(wspace=0.15, hspace=0.25)
    # plt.savefig(f"../outputs/figures/TR_mainshock_map_comp.png")
    plt.show()

In [ ]:
QTM_low_detection_threshold = pd.read_csv('../../catalogues/QTM/qtm_final_9.5dev.hypo.txt', delim_whitespace=True)

QTM_low_detection_threshold['DATETIME'] = pd.to_datetime(QTM_low_detection_threshold[['DAY', 'MONTH', 'YEAR', 'HOUR', 'MINUTE', 'SECOND']],
                                       format = '%d-%m-%Y %H:%M:%S')

In [ ]:
QTM_low_detection_threshold.loc[QTM_low_detection_threshold['EVENTID'].isin(my_extra_MDET_QTM_9_5['ID'])]

### Moving plots to new folder

In [ ]:
mainshock_folder_dict = {'MDET_extra_QTM_9_5':my_extra_MDET_QTM_9_5,
                         'MDET_missed_SCSN':missed_in_SCSN}

for key, item in mainshock_folder_dict.items():
    mainshock_IDs = list(item['ID'])
    print(len(mainshock_IDs))
    filenames_to_copy = [str(x) + '.png' for x in mainshock_IDs]

    source_folder = '../outputs/QTM_12/data_plots/'

    destination_folder = f'../outputs/QTM_12/data_plots/{key}/'
    Path(destination_folder).mkdir(parents=True, exist_ok=True)

    for filename in filenames_to_copy:
        source_file = os.path.join(source_folder, filename)
        
        if os.path.exists(source_file):
            destination_file = os.path.join(destination_folder, filename)
            
            shutil.copyfile(source_file, destination_file)
            print(f"File '{filename}' copied to '{destination_folder}'")
        else:
            print(f"File '{filename}' not found in '{source_folder}'")

## Moutote mainshocks

In [ ]:
QTM_12_FET = QTM_12_mainshocks.loc[QTM_12_mainshocks['Selection'].isin(['Both','FET'])].copy()
print(len(QTM_12_FET))
QTM_12_FET_box = seispy.restrict_catalogue_geographically(QTM_12_FET, region=low_Mc_region)
print(len(QTM_12_FET_box))
QTM_12_FET_box_no_10 = QTM_12_FET_box.loc[QTM_12_FET_box['n_local_cat_1yr']<10].copy()
print(len(QTM_12_FET_box_no_10))
QTM_12_FET_extra = QTM_12_FET_box.loc[(QTM_12_FET_box['In_Moutote']==False) & ~(QTM_12_FET_box['ID'].isin(QTM_12_FET_box_no_10['ID']))].copy()
print(len(QTM_12_FET_extra))
QTM_12_in_Moutote = QTM_12_mainshocks.loc[(QTM_12_mainshocks['In_Moutote']==True) & (QTM_12_mainshocks['Selection'].isin(['Both', 'FET']))].copy()
print(len(QTM_12_in_Moutote))
QTM_12_missed_FET = QTM_12_mainshocks.loc[(QTM_12_mainshocks['In_Moutote']==True) & ~(QTM_12_mainshocks['Selection'].isin(['Both', 'FET']))].copy()
print(len(QTM_12_missed_FET))
QTM_12_missed_FET

In [ ]:
QTM_12_FET_extra

In [ ]:
scale = 2
size_relation = (lambda x: (5*scale + np.exp(1.1*x))*scale)

size_dict = {'figsize_x': 8*scale, 
             'figsize_y': 7*scale,
             'textsize': 20*scale,
             'linewidth':3*scale,
             'edge_width':1/scale,
             'legend_markersize':10*scale,
             'point_alpha':0.9,
             'ec':'white',
             'fault_lw':2/scale,
             'fault_fc':'black',
             'label_pos':(-122.05, 31.65),
             'size':(lambda x: (5*scale + np.exp(1.1*x))*scale),
             'zorder':1
             }

params=size_dict

neither_relation = (lambda x: (5*scale + np.exp(1.1*x))/scale)
selection_dict = {'Both':{'zorder':11, 'color':plot_color_dict['teal'], 'size':size_relation, 'linewidth':0.5},
                    'FET':{'zorder':12, 'color':plot_color_dict['orange'], 'size':size_relation, 'linewidth':0.5},
                    'MDET':{'zorder':13, 'color':plot_color_dict['purple'], 'size':size_relation, 'linewidth':0.5},
                    'Neither':{'zorder':10, 'color':plot_color_dict['yellow'], 'size':neither_relation, 'linewidth':0.25}
                    }

labels = [letter + ')' for letter in alphabet[:4]]

box_alpha = 0.2
with plt.rc_context({'axes.titlesize': size_dict['textsize'],
                     'axes.labelsize': size_dict['textsize'], 
                     'xtick.labelsize':size_dict['textsize'],
                       'ytick.labelsize':size_dict['textsize'],
                       'legend.fontsize':size_dict['textsize']/2}):
    fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
    gs = fig.add_gridspec(3,3)

    ax2 = fig.add_subplot(gs[:2, :])
    ax4 = fig.add_subplot(gs[2, :])

    ax2.scatter(QTM_12_FET['geometry'].x, QTM_12_FET['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET['MAGNITUDE']), color=plot_color_dict['grey'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                label=f"All in QTM 12: {len(QTM_12_FET)}")
    ax2.scatter(QTM_12_FET_box['geometry'].x, QTM_12_FET_box['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_box['MAGNITUDE']), color=plot_color_dict['teal'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'],
                label=fr"In box: {len(QTM_12_FET_box)}")
    ax2.scatter(QTM_12_FET_box_no_10['geometry'].x, QTM_12_FET_box_no_10['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_box_no_10['MAGNITUDE']), color=plot_color_dict['purple'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'],
                label=f"<10 events yr prior: {len(QTM_12_FET_box_no_10)}")
    # ax2.scatter(QTM_12_FET_extra['geometry'].x, QTM_12_FET_extra['geometry'].y, 
    #             alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_extra['MAGNITUDE']), color=plot_color_dict['orange'],
    #             zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'],
    #             label=f"Extra: {len(QTM_12_FET_extra)}")
    ax2.scatter(QTM_12_missed_FET['geometry'].x, QTM_12_missed_FET['geometry'].y, 
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_missed_FET['MAGNITUDE']), color=plot_color_dict['pink'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'],
                label=f"Excluded: {len(QTM_12_missed_FET)}")
        
    # radius_km = 356.167877*1000	
    # MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
    # ax2.add_patch(MDET_circle)
    # MDET_circle = Circle((El_Mayor['geometry'].x, El_Mayor['geometry'].y), radius=radius_km, facecolor='None', edgecolor='purple', alpha=0.9, zorder=100)
    # ax1.add_patch(MDET_circle)

    ax4.scatter(seispy.datetime_to_decimal_year(QTM_12_FET['DATETIME']), QTM_12_FET['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET['MAGNITUDE']), color=plot_color_dict['grey'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(QTM_12_FET_box['DATETIME']), QTM_12_FET_box['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_box['MAGNITUDE']), color=plot_color_dict['teal'],
                zorder=params['zorder'], ec=size_dict['ec'], linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(QTM_12_FET_box_no_10['DATETIME']), QTM_12_FET_box_no_10['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_box_no_10['MAGNITUDE']), color=plot_color_dict['purple'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'])
    # ax4.scatter(seispy.datetime_to_decimal_year(QTM_12_FET_extra['DATETIME']), QTM_12_FET_extra['MAGNITUDE'],
    #             alpha=size_dict['point_alpha'], s=params['size'](QTM_12_FET_extra['MAGNITUDE']), color=plot_color_dict['orange'],
    #             zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'])
    ax4.scatter(seispy.datetime_to_decimal_year(QTM_12_missed_FET['DATETIME']), QTM_12_missed_FET['MAGNITUDE'],
                alpha=size_dict['point_alpha'], s=params['size'](QTM_12_missed_FET['MAGNITUDE']), color=plot_color_dict['pink'],
                zorder=params['zorder'], ec='white', linewidth=size_dict['edge_width'])

    ax4.set_ylabel('Magnitude')
    ax4.set_xlabel('Year')
    ax2.set_title('a)', loc='left')
    ax4.set_title('b)', loc='left')
    ax4.xaxis.set_ticks(range(2008, 2019,4))
    ax4.set_xticklabels(range(2008, 2019,4))

    box = low_Mc_region
    add_box_3857(box, ax=ax2, zorder=1, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['brown'])
    lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], ax=ax2, fontsize=size_dict['textsize'])
    
    ax2.axis(convert_extent_to_epsg3857([-122.2, -113.7, 31.25, 37.5]))

    cx.add_basemap(ax2, source=source)
    # fault_gdf.plot(color=size_dict['fault_fc'], linewidth=size_dict['fault_lw'], ax=ax2, zorder=1000)
    
    QTM_legend = [Line2D([], [], color=plot_color_dict['grey'], marker='o', markersize=size_dict['legend_markersize'], label=f"All in QTM 9.5: {len(my_MDET_QTM_9_5)}", linestyle='None'),
                Line2D([], [], color=plot_color_dict['teal'], marker='o', markersize=size_dict['legend_markersize'], label=fr"In box: {len(my_MDET_low_Mc_QTM_9_5)}", linestyle='None'),
                Line2D([], [], color=plot_color_dict['pink'], marker='o', markersize=size_dict['legend_markersize'], label=f"Outside box in SCSN: {len(missed_in_SCSN)}", linestyle='None'),
                # Line2D([], [], color=plot_color_dict['orange'], marker='o', markersize=size_dict['legend_markersize'], label=f"Extra: {len(my_extra_MDET_QTM_9_5)}", linestyle='None')
                ]

    # ax2.legend(handles=QTM_legend, loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
    ax2.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
    # pos = ax2.get_position()
    # new_pos = [pos.x0 - 0.7, pos.y0, pos.width, pos.height]
    # ax2.set_position(new_pos)
    # ax4.legend(handles=EM_legend, loc='lower center', bbox_to_anchor=(0.79, 0.59), ncol=1)
    plt.tight_layout()
    # fig.subplots_adjust(wspace=0.15, hspace=0.25)
    plt.savefig(f"../outputs/figures/Moutote_mainshock_map_comp.png")
    plt.show()

### The two mainshocks my FET method excludes

In [ ]:
QTM_12_missed_FET

In [ ]:
QTM_12_missed_FET['Moutote_excluded_by']

In [ ]:
for m in QTM_12_missed_FET.itertuples():
    l = ast.literal_eval(m.TR_excluded_by)
    print(len(l))

In [ ]:
cause_QTM = seispy.find_event_in_catalog(14612516, QTM_12)
cause_QTM

In [ ]:
seispy.calculate_distance_pyproj_vectorized(cause_QTM['LON'], cause_QTM['LAT'], QTM_12_missed_FET['LON'], QTM_12_missed_FET['LAT'])

In [ ]:
El_Mayor = merged_mainshocks_catalog.loc[merged_mainshocks_catalog['MAGNITUDE']>7].iloc[0]
seispy.calculate_distance_pyproj_vectorized(El_Mayor['LON'], El_Mayor['LAT'], QTM_12_missed_FET['LON'], QTM_12_missed_FET['LAT'])

In [ ]:
SCSN_missed_FET = SCSN_mainshocks.loc[SCSN_mainshocks['ID'].isin(QTM_12_missed_FET['ID'])].copy()
SCSN_missed_FET

In [ ]:
SCSN_missed_FET['Moutote_excluded_by']

In [ ]:
cause_SCSN = SCSN.loc[SCSN['ID'].isin([14612516, 14618772])].copy()
cause_SCSN

In [ ]:
for cause in cause_SCSN.itertuples():
    print(cause.ID, seispy.calculate_distance_pyproj_vectorized(cause.LON, cause.LAT, SCSN_missed_FET['LON'], SCSN_missed_FET['LAT']))

In [ ]:
def haversine_vectorised(lon1, lat1, lon2, lat2):
    """
    Returns the distance (km) from a point to an array of points using the haversine method
    """
    lon2, lat2 = np.array(lon2), np.array(lat2)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c 
    return km

print(haversine_vectorised(cause_SCSN['LON'].iloc[0], cause_SCSN['LAT'].iloc[0], SCSN_missed_FET['LON'], SCSN_missed_FET['LAT']),
      haversine_vectorised(cause_SCSN['LON'].iloc[1], cause_SCSN['LAT'].iloc[1], SCSN_missed_FET['LON'], SCSN_missed_FET['LAT'])
)

In [ ]:
# draw boxes as well as circles

mainshock_type_dict = {'QTM_12_missed_FET':{'file':QTM_12_missed_FET, 'fc':plot_color_dict['pink'], 'ec':'white', 'z':1, 'marker':'o', 'linewidth':2,
                                            'label':'Excluded by our FET (QTM)'},
                       'SCSN_missed_FET':{'file':SCSN_missed_FET, 'fc':plot_color_dict['pink'], 'ec':'white', 'z':1, 'marker':'s', 'linewidth':2,
                                          'label':'Excluded by our FET (SCSN)'},
                       'cause_QTM':{'file':cause_QTM, 'fc':plot_color_dict['grey'], 'ec':plot_color_dict['orange'], 'z':1, 'marker':'o', 'linewidth':5,
                                    'label':'Exclusion cause (QTM)'},
                       'cause_SCSN':{'file':cause_SCSN, 'fc':plot_color_dict['grey'], 'ec':plot_color_dict['teal'], 'z':1, 'marker':'s', 'linewidth':5,
                                     'label':'Exclusion cause (SCSN)'},
                       }

fig = plt.figure(figsize=(size_dict['figsize_x'],size_dict['figsize_y']), dpi=300)
ax2 = fig.add_subplot(111)

for key, item in mainshock_type_dict.items():
    file = item['file']
    ax2.scatter(file['geometry'].x, file['geometry'].y, marker=item['marker'], linewidth=item['linewidth'],
                alpha=size_dict['point_alpha'], s=params['size'](file['MAGNITUDE'])*3, color=item['fc'],
                zorder=item['z'], ec=item['ec'],
                label=f"{item['label']}")
    if 'cause' in key:
        if 'QTM' in key:
            fc = 'orange'
        elif 'SCSN' in key:
            fc='teal'
        for f in file.itertuples():
            # radius = 1e4
            radius = 11894.653217639774
            x,y = seispy.add_distance_to_position_haversine(file['LON'].iloc[0], file['LAT'].iloc[0], -10, 0)
            x_m, y_m = EPSG_transformer(np.array([x,y]))[0]
            radius = ((file['geometry'].x.iloc[0]-x_m)**2 + (file['geometry'].y.iloc[0]-y_m)**2)**0.5
            print(radius)
            circle_10km = Circle((f.geometry.x, f.geometry.y), radius=radius, facecolor='none', edgecolor=plot_color_dict[fc], alpha=0.8, #label='10 km radius',
                                  zorder=100, linewidth=item['linewidth'])
            ax2.add_patch(circle_10km)
            rect = plt.Rectangle((f.geometry.x - radius, f.geometry.y - radius), radius*2, radius*2,
                     linewidth=item['linewidth'], edgecolor=plot_color_dict[fc], facecolor='none')
            ax2.add_patch(rect)

box = low_Mc_region
add_box_3857(box, ax=ax2, zorder=1, linewidth=size_dict['linewidth']/2, alpha=0.9, color=plot_color_dict['brown'])
# lat_lon_tick_labels([-120, -116], [32, 34, 36, 38], ax=ax2, fontsize=size_dict['textsize'])

ax2.axis(convert_extent_to_epsg3857([-115.95, -115.65, 32.55, 32.8]))
# ax2.legend(loc='upper left', bbox_to_anchor=(1,1), fontsize=15)
ax2.legend(loc='lower left', fontsize=30).set_zorder(100)
# , label=r'low $M_{c} region'
ax2.set_xlabel('X (m)', fontsize=30)
ax2.set_xlabel('Y (m)', fontsize=30)

cx.add_basemap(ax2, source=source)
plt.tight_layout()
plt.savefig('../outputs/figures/Moutote_2_excluded.png')
plt.show()

#### Making a 10 km circle look like it's 10 km on a EPSG 3857 projection

In [ ]:
p = [cause_QTM['LON'], cause_QTM['LAT'], QTM_12_missed_FET['LON'], QTM_12_missed_FET['LAT']]
p = [p.iloc[0] for p in p]
p

In [ ]:
x,y = seispy.add_distance_to_position_haversine(cause_QTM['LON'], cause_QTM['LAT'], 10, 0)
x,y

In [ ]:
seispy.calculate_distance_pyproj_vectorized(cause_QTM['LON'], cause_QTM['LAT'], x, y)

In [ ]:
x1, y1, x2, y2 = cause_QTM['geometry'].x.iloc[0], cause_QTM['geometry'].y.iloc[0], QTM_12_missed_FET['geometry'].x.iloc[0], QTM_12_missed_FET['geometry'].y.iloc[0]
# x1, x2, y1, y2
((x1-x2)**2 + (y1-y2)**2)**0.5

In [ ]:
x_m, y_m = EPSG_transformer(np.array([x,y]))[0]
x_m, y_m

In [ ]:
((x1-x_m)**2 + (y1-y_m)**2)**0.5

In [ ]:
from pyproj import Geod
geod = Geod(ellps="WGS84")

# Define the coordinates
lon1, lat1 = -115.7851, 32.6843
lon2, lat2 = -115.8823, 32.72137

# Calculate forward and back azimuths, plus distance
azimuth1, azimuth2, distance = geod.inv(lon1, lat1, lon2, lat2)
print(f"Distance: {distance} meters")

In [ ]:
haversine_vectorised(cause_QTM['LON'].iloc[0], cause_QTM['LAT'].iloc[0], QTM_12_missed_FET['LON'], QTM_12_missed_FET['LAT'])
# seispy.haversine_vectorised(cause['LON'], cause['LAT'], QTM_12_missed_FET['LON'], QTM_12_missed_FET['LAT'])

### Moving data plots to new folders based on the sub-selections

In [ ]:
mainshock_folder_dict = {'FET_no_10':QTM_12_FET_box_no_10,
                         'FET_extra':QTM_12_FET_extra,
                         'FET_missed':QTM_12_missed_FET}

for key, item in mainshock_folder_dict.items():
    mainshock_IDs = list(item['ID'])
    print(len(mainshock_IDs))
    filenames_to_copy = [str(x) + '.png' for x in mainshock_IDs]

    source_folder = '../outputs/QTM_12/data_plots/'

    destination_folder = f'../outputs/QTM_12/data_plots/{key}/'
    Path(destination_folder).mkdir(parents=True, exist_ok=True)

    for filename in filenames_to_copy:
        source_file = os.path.join(source_folder, filename)
        
        if os.path.exists(source_file):
            destination_file = os.path.join(destination_folder, filename)
            
            shutil.copyfile(source_file, destination_file)
            print(f"File '{filename}' copied to '{destination_folder}'")
        else:
            print(f"File '{filename}' not found in '{source_folder}'")

# S1. Imaging basemaps

In [ ]:
extent = [-121.953, -114.026, 31.86317, 37.22333]
converted_extent = convert_extent_to_epsg3857(extent)
print("Converted extent in EPSG:3857:", converted_extent)

providers = cx.providers.flatten()
fig = plt.figure()
ax = fig.add_subplot(111)
extent = [-121.953, -114.026, 31.86317, 37.22333]
ax.axis(converted_extent)
cx.add_basemap(ax, source=providers['Esri.WorldTerrain'])
ax.set_title(name)
ax.set_axis_off()
plt.show()
# fig.savefig(f'../outputs/basemaps/{name}.png', bbox_inches="tight")

In [ ]:
providers = cx.providers.flatten()
selection = providers.keys()
# extent = (-12600000, -10300000, 1800000, 3800000)
banned_providers = []
for name in selection:
    if name not in banned_providers:
        try:
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.axis(converted_extent)
            cx.add_basemap(ax, source=providers[name])
            ax.set_title(name)
            ax.set_axis_off()
            plt.show()
            fig.savefig(f'../outputs/basemaps/{name}.png', bbox_inches="tight")
        except:
            banned_providers.append(name)
            print('probably the none type error')
    clear_output(wait=True)